In [279]:
import numpy as np
import os
import pandas as pd
from arcpy import env
from arcpy import TableToTable_conversion
import matplotlib.ticker as mtick

In [280]:
dirWork = os.getcwd()
dirData = os.path.join(dirWork, r'data')
dirTDM  = os.path.join(dirData, r'tdm\1_WF')
dirFactors   = os.path.join(dirWork, r'data-processed\factors')
dirResults   = os.path.join(dirWork, r'results')

csv_FacGeoFunGroupAreaTypeVolume_ToStationGroup = os.path.join(dirFactors, r'facgeo_fungroup_areatype_volume_to_station_group.csv')
csv_SegmentsMatchedToFactorGroups = os.path.join(dirFactors,'SegmentsMatchedToFactorGroups.csv')
csv_SeasonFactors = os.path.join(dirFactors, r'SeasonFactors.csv')
csv_SeasonFactors = os.path.join(dirFactors, r'SeasonFactors.csv')
csv_DOWFactors = os.path.join(dirFactors, r'DOWFactors.csv')
csv_TODFactors = os.path.join(dirFactors, r'TODFactors.csv')


shp_Forecasts = r"data\forecasts\Traffic-Volume-Historic-and-Forecast.shp"

In [281]:
df_SegmentsMatchedToFactorGroups = pd.read_csv(csv_SegmentsMatchedToFactorGroups)
display(df_SegmentsMatchedToFactorGroups)

,SEGID,AADT2019,ATYPE,FTGROUP,FACGEO,STATIONGROUP
0,0006_141.0,1517.0,1,Arterial,Statewide,COP
1,0006_146.9,1517.0,1,Arterial,Statewide,COP
2,0006_149.9,2441.0,1,Arterial,Statewide,COP
3,0006_150.6,2441.0,1,Arterial,Statewide,COP
4,0006_152.6,2417.0,1,Arterial,Statewide,COP
...,...,...,...,...,...,...
5344,0089_326.5,13767.0,3,Arterial,Statewide,COT
5345,2849_001.5,1096.0,2,Arterial,Statewide,COS
5346,MAG_6856,0.0,3,Arterial,Statewide,COT
5347,2845_001.0,3546.0,3,Arterial,Statewide,COT


# Run Settings

In [282]:
#set to true for first time to create CSVs, false to speed up for subsequent times
#process_TDMdbftocsv = True

#export results
export_results = True

# Define Scenarios

In [283]:
df_Scenarios = pd.DataFrame()

#define AdjCode for pivot when model is not related to forecast
#AADT will be adjusted based on difference between and forecast
#if ForecastScenarioID is note defined, then the associated forecast volume will be used for that year
#DBF don't include filename extension

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'Base19',
                                    'ScenarioName'   : '2019 Base',
                                    'ScenarioYear'   :  2019,
                                    'ScenarioGroup'  : 'Base',
                                    'SegSummaryCSV'  : 'v901_SE19_Net19_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'No'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'Base23',
                                    'ScenarioName'   : '2023 Base',
                                    'ScenarioYear'   :  2023,
                                    'ScenarioGroup'  : 'Base',
                                    'SegSummaryCSV'  : 'v901_SE23_Net23_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'Base19'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'TIP28',
                                    'ScenarioName'   : '2028 TIP Projects',
                                    'ScenarioYear'   :  2028,
                                    'ScenarioGroup'  : 'TIP',
                                    'SegSummaryCSV'  : 'v901_TIP_SE28_Net28_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'Base23'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'RTP32',
                                    'ScenarioName'   : '2032 Fiscally-Constrained',
                                    'ScenarioYear'   :  2032,
                                    'ScenarioGroup'  : 'RTP',
                                    'SegSummaryCSV'  : 'v901_RTP_SE32_Net32_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'TIP28',
                                    'AdjScenarioCode': None
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'RTP42',
                                    'ScenarioName'   : '2042 Fiscally-Constrained',
                                    'ScenarioYear'   :  2042,
                                    'ScenarioGroup'  : 'RTP',
                                    'SegSummaryCSV'  : 'v901_RTP_SE42_Net42_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'RTP32'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode' : 'RTP50',
                                    'ScenarioName' : '2050 Fiscally-Constrained',
                                    'ScenarioYear' :  2050,
                                    'ScenarioGroup': 'RTP',
                                    'SegSummaryCSV': 'v901_RTP_SE50_Net50_Summary_SEGID_Detailed.csv',
                                    'Process'      : 'No',
                                    'PrvScenarioCode': 'RTP42'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : '28NB32',
                                    'ScenarioName'   : '2032 SE on a 2028 Network',
                                    'ScenarioYear'   :  2032,
                                    'ScenarioGroup'  : 'NoBuild28',
                                    'AdjScenarioCode': 'RTP32',
                                    'SegSummaryCSV'  : 'v901_NoBuild_SE32_Net28_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'TIP28'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : '28NB42',
                                    'ScenarioName'   : '2042 SE on a 2028 Network',
                                    'ScenarioYear'   :  2042,
                                    'ScenarioGroup'  : 'NoBuild28',
                                    'AdjScenarioCode': 'RTP42',
                                    'SegSummaryCSV'  : 'v901_NoBuild_SE42_Net28_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': '28NB32'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : '28NB50',
                                    'ScenarioName'   : '2050 SE on a 2028 Network',
                                    'ScenarioYear'   :  2050,
                                    'ScenarioGroup'  : 'NoBuild28',
                                    'AdjScenarioCode': 'RTP50',
                                    'SegSummaryCSV'  : 'v901_NoBuild_SE50_Net28_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': '28NB42'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : '32NB42',
                                    'ScenarioName'   : '2042 SE on a 2032 Network',
                                    'ScenarioYear'   :  2042,
                                    'ScenarioGroup'  : 'NoBuild32',
                                    'AdjScenarioCode': 'RTP42',
                                    'SegSummaryCSV'  : 'v901_NoBuild_SE42_Net32_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'RTP32'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : '32NB50',
                                    'ScenarioName'   : '2050 SE on a 2032 Network',
                                    'ScenarioYear'   :  2050,
                                    'ScenarioGroup'  : 'NoBuild32',
                                    'AdjScenarioCode': 'RTP50',
                                    'SegSummaryCSV'  : 'v901_NoBuild_SE50_Net32_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': '32NB42'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : '42NB50',
                                    'ScenarioName'   : '2050 SE on a 2042 Network',
                                    'ScenarioYear'   :  2050,
                                    'ScenarioGroup'  : 'NoBuild42',
                                    'AdjScenarioCode': 'RTP50',
                                    'SegSummaryCSV'  : 'v901_NoBuild_SE50_Net42_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'RTP42'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'UF32',
                                    'ScenarioName'   : '2032 Needs-Based',
                                    'ScenarioYear'   :  2032,
                                    'ScenarioGroup'  : 'Needs',
                                    'AdjScenarioCode': 'RTP32',
                                    'SegSummaryCSV'  : 'v901_Needs_SE32_Net32_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'TIP28'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'UF42',
                                    'ScenarioName'   : '2042 Needs-Based',
                                    'ScenarioYear'   :  2042,
                                    'ScenarioGroup'  : 'Needs',
                                    'AdjScenarioCode': 'RTP42',
                                    'SegSummaryCSV'  : 'v901_Needs_SE42_Net42_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'UF32'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'UF50',
                                    'ScenarioName'   : '2050 Needs-Based',
                                    'ScenarioYear'   :  2050,
                                    'ScenarioGroup'  : 'Needs',
                                    'AdjScenarioCode': 'RTP50',
                                    'SegSummaryCSV'  : 'v901_Needs_SE50_Net50_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'UF42'
                                   },ignore_index=True)

#df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'UF32_MAG',
#                                    'ScenarioName'   : '2032 Needs-Based - MAG',
#                                    'ScenarioYear'   :  2032,
#                                    'ScenarioGroup'  : 'NeedsMAG',
#                                    'AdjScenarioCode': 'RTP32',
#                                    'SegSummaryCSV'  : 'v901_Needs_MAG_SE32_Net32_Summary_SEGID_Detailed.csv',
#                                    'Process'        : 'Yes',
#                                    'PrvScenarioCode': 'TIP28'
#                                   },ignore_index=True)
#
#df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'UF42_MAG',
#                                    'ScenarioName'   : '2042 Needs-Based - MAG',
#                                    'ScenarioYear'   :  2042,
#                                    'ScenarioGroup'  : 'NeedsMAG',
#                                    'AdjScenarioCode': 'RTP42',
#                                    'SegSummaryCSV'  : 'v901_Needs_MAG_SE42_Net42_Summary_SEGID_Detailed.csv',
#                                    'Process'        : 'Yes',
#                                    'PrvScenarioCode': 'UF32_MAG'
#                                   },ignore_index=True)
#
df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'UF50_MAG',
                                    'ScenarioName'   : '2050 Needs-Based - MAG',
                                    'ScenarioYear'   :  2050,
                                    'ScenarioGroup'  : 'NeedsMAG',
                                    'AdjScenarioCode': 'RTP50',
                                    'SegSummaryCSV'  : 'v901_Needs_MAG_SE50_Net50_Summary_SEGID_Detailed.csv',
                                    'Process'        : 'Yes',
                                    'PrvScenarioCode': 'UF50'
                                   },ignore_index=True)


#all scenarios will be read in to allow for pivoting if needed
df_Scenarios.index.name = 'ScenarioID'
df_Scenarios['ScenarioYear'] = pd.to_numeric(df_Scenarios['ScenarioYear'], downcast='integer')
df_Scenarios

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Base19,2019 Base,2019,Base,v901_SE19_Net19_Summary_SEGID_Detailed.csv,No,NaN,NaN
1,Base23,2023 Base,2023,Base,v901_SE23_Net23_Summary_SEGID_Detailed.csv,Yes,Base19,NaN
2,TIP28,2028 TIP Projects,2028,TIP,v901_TIP_SE28_Net28_Summary_SEGID_Detailed.csv,Yes,Base23,NaN
3,RTP32,2032 Fiscally-Constrained,2032,RTP,v901_RTP_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,NaN
4,RTP42,2042 Fiscally-Constrained,2042,RTP,v901_RTP_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,RTP32,NaN
...,...,...,...,...,...,...,...,...
11,42NB50,2050 SE on a 2042 Network,2050,NoBuild42,v901_NoBuild_SE50_Net42_Summary_SEGID_Detailed...,Yes,RTP42,RTP50
12,UF32,2032 Needs-Based,2032,Needs,v901_Needs_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,RTP32
13,UF42,2042 Needs-Based,2042,Needs,v901_Needs_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,UF32,RTP42


In [284]:
df_Scenarios_toProcess = df_Scenarios[df_Scenarios['Process']=='Yes']
df_Scenarios_toProcess

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
1,Base23,2023 Base,2023,Base,v901_SE23_Net23_Summary_SEGID_Detailed.csv,Yes,Base19,NaN
2,TIP28,2028 TIP Projects,2028,TIP,v901_TIP_SE28_Net28_Summary_SEGID_Detailed.csv,Yes,Base23,NaN
3,RTP32,2032 Fiscally-Constrained,2032,RTP,v901_RTP_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,NaN
4,RTP42,2042 Fiscally-Constrained,2042,RTP,v901_RTP_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,RTP32,NaN
6,28NB32,2032 SE on a 2028 Network,2032,NoBuild28,v901_NoBuild_SE32_Net28_Summary_SEGID_Detailed...,Yes,TIP28,RTP32
...,...,...,...,...,...,...,...,...
11,42NB50,2050 SE on a 2042 Network,2050,NoBuild42,v901_NoBuild_SE50_Net42_Summary_SEGID_Detailed...,Yes,RTP42,RTP50
12,UF32,2032 Needs-Based,2032,Needs,v901_Needs_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,RTP32
13,UF42,2042 Needs-Based,2042,Needs,v901_Needs_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,UF32,RTP42


In [285]:
df_Scenarios_toAdjust = df_Scenarios_toProcess[df_Scenarios_toProcess['AdjScenarioCode'].notnull()]
df_Scenarios_toAdjust

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
6,28NB32,2032 SE on a 2028 Network,2032,NoBuild28,v901_NoBuild_SE32_Net28_Summary_SEGID_Detailed...,Yes,TIP28,RTP32
7,28NB42,2042 SE on a 2028 Network,2042,NoBuild28,v901_NoBuild_SE42_Net28_Summary_SEGID_Detailed...,Yes,28NB32,RTP42
8,28NB50,2050 SE on a 2028 Network,2050,NoBuild28,v901_NoBuild_SE50_Net28_Summary_SEGID_Detailed...,Yes,28NB42,RTP50
9,32NB42,2042 SE on a 2032 Network,2042,NoBuild32,v901_NoBuild_SE42_Net32_Summary_SEGID_Detailed...,Yes,RTP32,RTP42
10,32NB50,2050 SE on a 2032 Network,2050,NoBuild32,v901_NoBuild_SE50_Net32_Summary_SEGID_Detailed...,Yes,32NB42,RTP50
11,42NB50,2050 SE on a 2042 Network,2050,NoBuild42,v901_NoBuild_SE50_Net42_Summary_SEGID_Detailed...,Yes,RTP42,RTP50
12,UF32,2032 Needs-Based,2032,Needs,v901_Needs_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,RTP32
13,UF42,2042 Needs-Based,2042,Needs,v901_Needs_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,UF32,RTP42
14,UF50,2050 Needs-Based,2050,Needs,v901_Needs_SE50_Net50_Summary_SEGID_Detailed.csv,Yes,UF42,RTP50


In [286]:
#define functions to get scenario data
def idFromCode (sCode):
    return df_Scenarios.index[df_Scenarios['ScenarioCode'] == sCode].tolist()[0]
def yrFromCode (sCode):
    return df_Scenarios[df_Scenarios['ScenarioCode'] == sCode].iloc[0]['ScenarioYear']
def codeFromID (sID):
    return df_Scenarios.loc[sID]['ScenarioCode']
def yearFromID (sID):
    return df_Scenarios.loc[sID]['ScenarioYear']

#quick tests of functions
#display(idFromCode('50RTP'))
#display(yrFromCode('50RTP'))
#display(codeFromID(4))
#display(yearFromID(4))
#display(df_Scenarios.loc[7]['ScenarioCode'])

# Define Desired V/C Groupings

In [287]:
#season groups

df_VCGroup_Seasons = pd.DataFrame()

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Ann',
                                                'SeasonName'    : 'Annual',
                                                'SEASONGROUP'   : 'S00-Ann',
                                                'SeasonType'    : 'Season'
                                               },ignore_index=True)

#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Win',
#                                                'SeasonName'    : 'Winter',
#                                                'SEASONGROUP'   : 'S01-Win',
#                                                'SeasonType'    : 'Season'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Spr',
#                                                'SeasonName'    : 'Spring',
#                                                'SEASONGROUP'   : 'S02-Spr',
#                                                'SeasonType'    : 'Season'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Sum',
#                                                'SeasonName'    : 'Summer',
#                                                'SEASONGROUP'   : 'S03-Sum',
#                                                'SeasonType'    : 'Season'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Fal',
#                                                'SeasonName'    : 'Fall',
#                                                'SEASONGROUP'   : 'S04-Fal',
#                                                'SeasonType'    : 'Season'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Jan',
#                                                'SeasonName'    : 'January',
#                                                'SEASONGROUP'   : 'M01-Jan',
#                                                'SeasonType'    : 'Month',
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Feb',
#                                                'SeasonName'    : 'February',
#                                                'SEASONGROUP'   : 'M02-Feb',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Mar',
#                                                'SeasonName'    : 'March',
#                                                'SEASONGROUP'   : 'M03-Mar',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Apr',
#                                                'SeasonName'    : 'April',
#                                                'SeasonGroup'   : 'M04-Apr',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'May',
#                                                'SeasonName'    : 'May',
#                                                'SEASONGROUP'   : 'M05-May',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Jun',
#                                                'SeasonName'    : 'June',
#                                                'SEASONGROUP'   : 'M06-Jun',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Jul',
#                                                'SeasonName'    : 'July',
#                                                'SEASONGROUP'   : 'M07-Jul',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Aug',
#                                                'SeasonName'    : 'August',
#                                                'SEASONGROUP'   : 'M08-Aug',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Sep',
#                                                'SeasonName'    : 'September',
#                                                'SEASONGROUP'   : 'M09-Sep',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Oct',
#                                                'SeasonName'    : 'October',
#                                                'SEASONGROUP'   : 'M10-Oct',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Nov',
#                                                'SeasonName'    : 'November',
#                                                'SEASONGROUP'   : 'M11-Nov',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)
#
#df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Dec',
#                                                'SeasonName'    : 'December',
#                                                'SEASONGROUP'   : 'M12-Dec',
#                                                'SeasonType'    : 'Month'
#                                               },ignore_index=True)

display(df_VCGroup_Seasons)

#day-of-week groups

df_VCGroup_DOWPkHr = pd.DataFrame()

df_VCGroup_DOWPkHr = df_VCGroup_DOWPkHr.append({'DOWPkCode' : 'WkAM',
                                                'DOWPkName' : 'AM - Weekday',
                                                'DOWGROUP'  : '1-Weekday (Tu-Th)',
                                                'PERIOD'    : 'AM',
                                                'HRPCTOF'   : 'Prd' #calculate peak Hr using Prd volume
                                               },ignore_index=True)

df_VCGroup_DOWPkHr = df_VCGroup_DOWPkHr.append({'DOWPkCode' : 'WkMD',
                                                'DOWPkName' : 'Midday - Weekday',
                                                'DOWGROUP'  : '1-Weekday (Tu-Th)',
                                                'PERIOD'    : 'MD',
                                                'HRPCTOF'   : 'Prd' #calculate peak Hr using Prd volume
                                               },ignore_index=True)

df_VCGroup_DOWPkHr = df_VCGroup_DOWPkHr.append({'DOWPkCode' : 'WkPM',
                                                'DOWPkName' : 'PM - Weekday',
                                                'DOWGROUP'  : '1-Weekday (Tu-Th)',
                                                'PERIOD'    : 'PM',
                                                'HRPCTOF'   : 'Prd' #calculate peak Hr using Prd volume
                                               },ignore_index=True)

#df_VCGroup_DOWPkHr = df_VCGroup_DOWPkHr.append({'DOWPkCode' : 'FrPM',
#                                                'DOWPkName' : 'PM - Friday',
#                                                'DOWGROUP'  : '5-Friday',
#                                                'PERIOD'    : 'PM',
#                                                'HRPCTOF'   : 'DY' #calculate peak Hr using DY volume
#                                               },ignore_index=True)
#
#df_VCGroup_DOWPkHr = df_VCGroup_DOWPkHr.append({'DOWPkCode' : 'SaDY',
#                                                'DOWPkName' : 'Saturday',
#                                                'DOWGROUP'  : '6-Saturday',
#                                                'PERIOD'    : 'DY',
#                                                'HRPCTOF'   : 'DY' #calculate peak Hr using DY volume
#                                               },ignore_index=True)
#
#df_VCGroup_DOWPkHr = df_VCGroup_DOWPkHr.append({'DOWPkCode' : 'SuDY',
#                                                'DOWPkName' : 'Sunday',
#                                                'DOWGROUP'  : '7-Sunday',
#                                                'PERIOD'    : 'DY',
#                                                'HRPCTOF'   : 'DY' #calculate peak Hr using DY volume
#                                               },ignore_index=True)

display(df_VCGroup_DOWPkHr)

#directional factor when using DY volumes for peak Hr calcs
#taken from HCM 2000 Exhibit 9-2
dDYDirectionalFactor = 0.6

#directional factor not used in Prd volume calcs, since those include directionality

,SeasonCode,SeasonName,SEASONGROUP,SeasonType
0,Ann,Annual,S00-Ann,Season


,DOWPkCode,DOWPkName,DOWGROUP,PERIOD,HRPCTOF
0,WkAM,AM - Weekday,1-Weekday (Tu-Th),AM,Prd
1,WkMD,Midday - Weekday,1-Weekday (Tu-Th),MD,Prd
2,WkPM,PM - Weekday,1-Weekday (Tu-Th),PM,Prd


In [288]:
import itertools

#create df with all possible season and day-of-weak/peak groups
df_VCGroups = pd.DataFrame(list(itertools.product(df_VCGroup_Seasons['SeasonCode'],df_VCGroup_DOWPkHr['DOWPkCode'])),columns=['SeasonCode','DOWPkCode'])

df_VCGroups = pd.DataFrame(df_VCGroups)
pd.set_option('display.max_rows', df_VCGroups.shape[0]+1)

#display(df_VCGroups)

#add season data
df_VCGroups = pd.DataFrame.merge(df_VCGroups, df_VCGroup_Seasons, on='SeasonCode')

#add dow-peak data
df_VCGroups = pd.DataFrame.merge(df_VCGroups, df_VCGroup_DOWPkHr, on='DOWPkCode')

#set id name
df_VCGroups.index.name = 'VCGroupID'

df_VCGroups['VCGroupCode'] = df_VCGroups['SeasonCode'] + df_VCGroups['DOWPkCode']

df_VCGroups = df_VCGroups.drop(columns=['SeasonCode','DOWPkCode','SeasonName','DOWPkName'])

display(df_VCGroups)

,SEASONGROUP,SeasonType,DOWGROUP,PERIOD,HRPCTOF,VCGroupCode
VCGroupID,,,,,,
0,S00-Ann,Season,1-Weekday (Tu-Th),AM,Prd,AnnWkAM
1,S00-Ann,Season,1-Weekday (Tu-Th),MD,Prd,AnnWkMD
2,S00-Ann,Season,1-Weekday (Tu-Th),PM,Prd,AnnWkPM


# TDM SEGID Detail

In [289]:
#column names from TDM
nameSegID      = 'SEGID'
nameFG         = 'FUNCGROUP'
nameLinks      = 'LINKS'
nameLanes      = 'LANES'
nameVol        = 'VOL'
nameCap1HL     = 'CAP1HL'
nameAT         = 'AREATYPE'
nameFT         = 'FT'
nameFTClass    = 'FTCLASS'
nameFGFac      = 'FGFAC'
namePrdFac     = 'PRDFAC'
nameDYVOL      = 'DY_VOL'
nameDirections = 'DIRECTIONS'
nameDFac       = 'DFAC'
nameMDPct      = 'MDPCT'
nameHVPct      = 'HVPCT'
nameSegTotal   = 'SegDYVol_TDM'

#column for scenario year
nameSID = 'ScenarioID'
nameS =   'ScenarioName'
nameSYr = 'ScenarioYear'

#additional column name variables
nameLT = 'LT'
nameMD = 'MD'
nameHV = 'HV'

nameVolType = 'VolType'
namePrd     = 'Prd'
nameCapPer  = 'CapPer'
nameCap1Hr  = 'Cap1Hr'
nameDir     = 'Dir'

#value for functional group (FG) total row, so that it can be removed
nameFGTotalValue = 'Total'

#arrays for values by direction

aNameDir1Columns   = ['D1_'+nameFT,'D1_'+nameFTClass,'D1_'+nameLanes,'D1_'+nameCap1HL]
aNameDir2Columns   = ['D2_'+nameFT,'D2_'+nameFTClass,'D2_'+nameLanes,'D2_'+nameCap1HL]
aNameDirColumns    = [      nameFT,      nameFTClass,      nameLanes,      nameCap1HL]    
aNameDirIndexNoDir = [nameSegID,nameFG]
aNameDirIndexWiDir = [nameSegID,nameFG,nameDir]

#arrays for values by Prd
aNamePrdFac = ['DY_PRDFAC','AM_PRDFAC','MD_PRDFAC','PM_PRDFAC','EV_PRDFAC'] #add DY_PRDFAC = 1 later in script for calc ease

#arrays for values by direction, Prd
aNameDirPrdDFac      = ['D1_DY_'+nameDFac     ,'D1_AM_'+nameDFac     ,'D1_MD_'+nameDFac     ,'D1_PM_'+nameDFac     ,'D1_EV_'+nameDFac     ,'D2_DY_'+nameDFac     ,'D2_AM_'+nameDFac     ,'D2_MD_'+nameDFac     ,'D2_PM_'+nameDFac     ,'D2_EV_'+nameDFac     ]
aNameDirPrdMDPct = ['D1_DY_'+nameMDPct,'D1_AM_'+nameMDPct,'D1_MD_'+nameMDPct,'D1_PM_'+nameMDPct,'D1_EV_'+nameMDPct,'D2_DY_'+nameMDPct,'D2_AM_'+nameMDPct,'D2_MD_'+nameMDPct,'D2_PM_'+nameMDPct,'D2_EV_'+nameMDPct]
aNameDirPrdHVPct = ['D1_DY_'+nameHVPct,'D1_AM_'+nameHVPct,'D1_MD_'+nameHVPct,'D1_PM_'+nameHVPct,'D1_EV_'+nameHVPct,'D2_DY_'+nameHVPct,'D2_AM_'+nameHVPct,'D2_MD_'+nameHVPct,'D2_PM_'+nameHVPct,'D2_EV_'+nameHVPct]

indexTotal             = [nameSegID]
indexMelt              = [nameSegID,nameFG]
#indexDY             = [nameSegID,nameFG,nameCap1HL,nameLinks,nameLanes,nameAT,nameFT]

#Hrs within Prd, for capacity
dPrdDuration = {
     'PERIOD': ['AM','MD','PM','EV','DY'],
     'PRDHRS': [   3,   6,   3,  12,     24]
     
}
df_PrdDuration = pd.DataFrame(dPrdDuration)

In [290]:
dfs_TDM =[]
dfs_TDM_Total = []

#all scenarios will be read in to allow for pivoting

for (idx, row) in df_Scenarios.iterrows():

    print(row.loc[nameS] + ' CSV Processing')

    csvFileName = os.path.join(dirTDM, row.SegSummaryCSV)

    df_TDM_import = pd.read_csv(csvFileName)
    df_TDM_import[nameFG] = df_TDM_import[nameFG].str.strip() #strip leading and trailing spaces
    
    #totals for DY segment volume
    df_TDM_SegDYVol = df_TDM_import[df_TDM_import[nameFG] == nameFGTotalValue].copy()
    df_TDM_SegDYVol = df_TDM_SegDYVol[[nameSegID,nameDYVOL]]
    df_TDM_SegDYVol = df_TDM_SegDYVol.rename(columns={nameDYVOL: nameSegTotal})
    
    dfs_TDM_Total.append(df_TDM_SegDYVol[[nameSegID,nameSegTotal]])
    
    #display(df_TDM_SegDYVol)
    
    #Remove rows with no links and total
    df_TDM_import         = df_TDM_import[df_TDM_import[nameLinks] > 0                ]
    df_TDM_import         = df_TDM_import[df_TDM_import[nameFG]    != nameFGTotalValue]

    #data with segment level values
    df_TDM_Seg = pd.DataFrame.merge(df_TDM_import[[nameSegID,nameFG,nameAT,nameFGFac]],df_TDM_SegDYVol,on=nameSegID)    
    
    #normalize data with direction fields
    df_Dir1Columns          = df_TDM_import[aNameDirIndexNoDir + aNameDir1Columns].copy()
    df_Dir1Columns.columns  = (aNameDirIndexNoDir + aNameDirColumns)
    df_Dir1Columns[nameDir] = 'D1'
    
    df_Dir2Columns          = df_TDM_import[aNameDirIndexNoDir + aNameDir2Columns].copy()
    df_Dir2Columns.columns  = (aNameDirIndexNoDir + aNameDirColumns)
    df_Dir2Columns[nameDir] = 'D2'
    
    #display(df_Dir1Columns)
    #display(df_Dir2Columns)
    
    df_TDM_Dir = pd.concat([df_Dir1Columns,df_Dir2Columns])
    #display(df_Dir)
    
    #normalize data with Prd fields
    df_TDM_import['DY_PRDFAC'] = 1
    df_TDM_PrdFac = pd.melt(df_TDM_import, id_vars=indexMelt, value_vars=aNamePrdFac)

    #directional factors
    df_TDM_PrdFac[namePrd] = df_TDM_PrdFac['variable'].str.get(0) + df_TDM_PrdFac['variable'].str.get(1)
    df_TDM_PrdFac          = df_TDM_PrdFac.rename(columns={'value': namePrdFac})
    df_TDM_PrdFac          = df_TDM_PrdFac.drop(columns=['variable'])
    
    df_TDM_Prd = df_TDM_PrdFac
    
    #display(df_TDM_Prd)
    
    #normalize data with dir/Prd fields
    df_TDM_DFac      = pd.melt(df_TDM_import, id_vars=indexMelt, value_vars=aNameDirPrdDFac     )
    df_TDM_MDPct = pd.melt(df_TDM_import, id_vars=indexMelt, value_vars=aNameDirPrdMDPct)
    df_TDM_HVPct = pd.melt(df_TDM_import, id_vars=indexMelt, value_vars=aNameDirPrdHVPct)

    #directional factors
    df_TDM_DFac[nameDir] = df_TDM_DFac['variable'].str.get(0) + df_TDM_DFac['variable'].str.get(1)
    df_TDM_DFac[namePrd] = df_TDM_DFac['variable'].str.get(3) + df_TDM_DFac['variable'].str.get(4)
    df_TDM_DFac          = df_TDM_DFac.rename(columns={'value': nameDFac})
    df_TDM_DFac          = df_TDM_DFac.drop(columns=['variable'])
    
    #display(df_TDM_DFac)
    
    #medium truck Pcts
    df_TDM_MDPct[nameDir] = df_TDM_MDPct['variable'].str.get(0) + df_TDM_MDPct['variable'].str.get(1)
    df_TDM_MDPct[namePrd] = df_TDM_MDPct['variable'].str.get(3) + df_TDM_MDPct['variable'].str.get(4)
    df_TDM_MDPct          = df_TDM_MDPct.rename(columns={'value': nameMDPct})
    df_TDM_MDPct          = df_TDM_MDPct.drop(columns=['variable'])
    
    #display(df_TDM_MDPct)    
    
    #medium truck Pcts
    df_TDM_HVPct[nameDir] = df_TDM_HVPct['variable'].str.get(0) + df_TDM_HVPct['variable'].str.get(1)
    df_TDM_HVPct[namePrd] = df_TDM_HVPct['variable'].str.get(3) + df_TDM_HVPct['variable'].str.get(4)
    df_TDM_HVPct          = df_TDM_HVPct.rename(columns={'value': nameHVPct})
    df_TDM_HVPct          = df_TDM_HVPct.drop(columns=['variable'])
    
    #display(df_TDM_HVPct)
    
    #merge dir/Prd dataframes
    df_TDM_DirPrd = pd.DataFrame.merge(df_TDM_DFac  ,df_TDM_MDPct,on=(nameSegID,nameFG,nameDir,namePrd))
    df_TDM_DirPrd = pd.DataFrame.merge(df_TDM_DirPrd,df_TDM_HVPct,on=(nameSegID,nameFG,nameDir,namePrd))
    
    #display(df_DirPrd)
    
    df_TDM_byFGDirPrd = pd.DataFrame.merge(df_TDM_Seg       ,df_TDM_Dir   ,on=(nameSegID,nameFG)) #merge direction
    df_TDM_byFGDirPrd = pd.DataFrame.merge(df_TDM_byFGDirPrd,df_TDM_Prd   ,on=(nameSegID,nameFG)) #merge Prd
    df_TDM_byFGDirPrd = pd.DataFrame.merge(df_TDM_byFGDirPrd,df_TDM_DirPrd,on=(nameSegID,nameFG,namePrd,nameDir)) #merge Prd/direction

    #calculate DY directional for peak-Hr-Pct-of-DY calcs
    #assumes when DY volumes is used, TDM directional factor not valid, eg. weekends    
    #both directions use same directional factor
    df_TDM_byFGDirPrd.loc[(df_TDM_byFGDirPrd[namePrd] == 'DY'), nameDFac] = dDYDirectionalFactor
    df_TDM_byFGDirPrd.loc[(df_TDM_byFGDirPrd[namePrd] == 'DY'), namePrd] = 'DY'
        
    df_TDM_byFGDirPrd = df_TDM_byFGDirPrd[[nameSegID,nameAT,nameSegTotal,nameFG,nameFGFac,nameDir,nameLanes,nameFT,nameFTClass,nameCap1HL,nameDFac,namePrd,namePrdFac,nameMDPct,nameHVPct]]
    #display(df_TDM_byFGDirPrd)

    dfs_TDM.append(df_TDM_byFGDirPrd)

    df_Check = df_TDM_byFGDirPrd.copy()
    df_Check['FactorsCombined'] = df_Check[nameFGFac] * df_Check[nameDFac] * df_Check[namePrdFac]
    
    #The sum of all factors should 2.2 (1.0 for non-total and 2xDFac(0.6) for totals, add little extra on each side for rounding
    df_Check = df_Check.groupby(indexTotal).agg({'FactorsCombined':[np.sum]})
    df_Check.columns = df_Check.columns.droplevel(1)
    df_Check = df_Check[((df_Check['FactorsCombined'] > 2.2003) | (df_Check['FactorsCombined'] < 2.1997) & (df_Check['FactorsCombined'] != 0))]
    display(print(row.loc[nameS] + ' Checks:'))
    display(df_Check)

print('Done')

2019 Base CSV Processing


2019 Base Checks:


None

,FactorsCombined
SEGID,


2023 Base CSV Processing
2023 Base Checks:


...

,FactorsCombined
SEGID,


2050 Needs-Based - MAG CSV Processing
2050 Needs-Based - MAG Checks:


None

,FactorsCombined
SEGID,


Done


In [291]:
dfTemp = dfs_TDM[8][dfs_TDM[8]['SEGID']=='MAG_6242']
dfTemp

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT


In [292]:
dfTemp_Total = dfs_TDM_Total[8][dfs_TDM_Total[8]['SEGID']=='0145_001.4']
dfTemp_Total

,SEGID,SegDYVol_TDM
4560,0145_001.4,71963.0


In [293]:
df_Scenarios_toProcess

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
1,Base23,2023 Base,2023,Base,v901_SE23_Net23_Summary_SEGID_Detailed.csv,Yes,Base19,NaN
2,TIP28,2028 TIP Projects,2028,TIP,v901_TIP_SE28_Net28_Summary_SEGID_Detailed.csv,Yes,Base23,NaN
...,...,...,...,...,...,...,...,...
14,UF50,2050 Needs-Based,2050,Needs,v901_Needs_SE50_Net50_Summary_SEGID_Detailed.csv,Yes,UF42,RTP50
15,UF50_MAG,2050 Needs-Based - MAG,2050,NeedsMAG,v901_Needs_MAG_SE50_Net50_Summary_SEGID_Detail...,Yes,UF50,RTP50


In [294]:
df_Scenarios_toAdjust

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
6,28NB32,2032 SE on a 2028 Network,2032,NoBuild28,v901_NoBuild_SE32_Net28_Summary_SEGID_Detailed...,Yes,TIP28,RTP32
7,28NB42,2042 SE on a 2028 Network,2042,NoBuild28,v901_NoBuild_SE42_Net28_Summary_SEGID_Detailed...,Yes,28NB32,RTP42
...,...,...,...,...,...,...,...,...
14,UF50,2050 Needs-Based,2050,Needs,v901_Needs_SE50_Net50_Summary_SEGID_Detailed.csv,Yes,UF42,RTP50
15,UF50_MAG,2050 Needs-Based - MAG,2050,NeedsMAG,v901_Needs_MAG_SE50_Net50_Summary_SEGID_Detail...,Yes,UF50,RTP50


In [295]:
#calculate adjustment for forecast for those non-RTP scenarios

dfs_ForecastAdj = []

for (idx, row) in df_Scenarios_toAdjust.iterrows():

    #print(row.loc[nameS] + ' CSV Importing')
    
    df_base = dfs_TDM_Total[idFromCode(row.loc['AdjScenarioCode'])]
    df_base.columns = (['SEGID','DYVolBase'])
    df_scenario  = dfs_TDM_Total[idFromCode(row.loc['ScenarioCode'])]
    df_scenario.columns = (['SEGID','DYVolScenario'])
    
    #display(df_base)
    #display(df_scenario)
    
    df_ForecastAdj = pd.DataFrame.merge(df_scenario,df_base,on=nameSegID,how='left')
    df_ForecastAdj['DYVolAdj'] = np.where(df_ForecastAdj['DYVolBase'] > 0,
                                        (df_ForecastAdj['DYVolScenario'] - df_ForecastAdj['DYVolBase']) / df_ForecastAdj['DYVolBase'],
                                        0)  # You can adjust this value to 'np.nan' or another appropriate value if zero is not suitable
    
    df_ForecastAdj.to_csv(os.path.join(dirResults,'ForecastAdj_' + row.loc['ScenarioCode'] + '.csv'))
    
    #display(df_ForecastAdj)
    
    dfs_ForecastAdj.append(df_ForecastAdj)

In [296]:
dfs_ForecastAdj[1][dfs_ForecastAdj[1]['SEGID']=='MAG_6242']

,SEGID,DYVolScenario,DYVolBase,DYVolAdj


# AADT and AvgAnnualPrdVol Forecasts (Segments, Forecasts, TDM)

In [297]:
#import arcgis libraries
from arcgis.gis import *
gis = GIS()
                        
#import into spatially-enabled DataFrame
sdf_Forecasts = pd.DataFrame.spatial.from_featureclass(shp_Forecasts)
sdf_Forecasts

,FID,Id,SEGID,ROUTE,BMP,EMP,DISTANCE,CO_FIPS,SUBAREAID,PLANAREA,F2019,F2023,F2028,F2032,F2042,F2050,CH19TO50,CH19TO23,CH23TO28,CH28TO32,CH32TO42,CH42TO50,AADTSTN,AADT2022,AADT2021,AADT2020,AADT2019,AADT2018,AADT2017,AADT2016,AADT2015,AADT2014,AADT2013,AADT2012,AADT2011,AADT2010,AADT2009,AADT2008,AADT2007,AADT2006,AADT2005,AADT2004,AADT2003,AADT2002,AADT2001,AADT2000,AADT1999,AADT1998,AADT1997,AADT1996,AADT1995,AADT1994,AADT1993,AADT1992,AADT1991,AADT1990,AADT1989,AADT1988,AADT1987,AADT1986,AADT1985,AADT1984,AADT1983,AADT1982,AADT1981,SUTRK2022,CUTRK2022,SUTRK2021,CUTRK2021,SUTRK2020,CUTRK2020,SUTRK2019,CUTRK2019,SUTRK2018,CUTRK2018,SUTRK2017,CUTRK2017,SUTRK2016,CUTRK2016,SUTRK2015,CUTRK2015,SUTRK2014,CUTRK2014,SUTRK2013,CUTRK2013,SUTRK2012,CUTRK2012,SUTRK2011,CUTRK2011,SUTRK2010,CUTRK2010,CCSGROUP19,FAC_MON,FAC_TUE,FAC_WED,FAC_THU,FAC_FRI,FAC_SAT,FAC_SUN,FAC_WDAVG,FAC_WEAVG,FAC_WEMAX,FAC_SPR,FAC_SUM,FAC_FAL,FAC_WIN,FAC_JAN,FAC_FEB,FAC_MAR,FAC_APR,FAC_MAY,FAC_JUN,FAC_JUL,FAC_AUG,FAC_SEP,FAC_OCT,FAC_NOV,FAC_DEC,FAC_MAXMO,FAC_MAX,FACMANADJ,SHAPE
0,0,0,0023_000.0,0023,0.000,1.564,1.585102,5.0,2.0,Cache,1969.0,2000.0,2400.0,2600.0,2700.0,3100.0,1131.0,31.0,400.0,200.0,100.0,400.0,,0.0,0.0,0.0,1969.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,,,0.0,0.0,,,,,,,0.0,,,,,,COS,1.0517,1.0744,1.0878,1.1132,1.1558,0.8742,0.6429,1.0918,0.7585,0.8742,1.0107,1.0575,1.0324,0.8994,0.8600,0.8982,0.9669,1.0119,1.0534,1.0636,1.0366,1.0724,1.0607,1.0468,0.9896,0.9399,8.0,1.0724,0.0,"{""paths"": [[[422233.4001000002, 4607564.0001],..."
1,1,0,0023_001.6,0023,1.564,2.146,0.580084,5.0,2.0,Cache,2955.0,2900.0,3200.0,3400.0,3300.0,3500.0,545.0,-55.0,300.0,200.0,-100.0,200.0,,0.0,0.0,0.0,2955.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,,,0.0,0.0,,,,,,,0.0,,,,,,COS,1.0517,1.0744,1.0878,1.1132,1.1558,0.8742,0.6429,1.0918,0.7585,0.8742,1.0107,1.0575,1.0324,0.8994,0.8600,0.8982,0.9669,1.0119,1.0534,1.0636,1.0366,1.0724,1.0607,1.0468,0.9896,0.9399,8.0,1.0724,0.0,"{""paths"": [[[422182.7999999998, 4610094.1], [4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5836,5836,0,WFRC_8475,WFRC,0.000,0.000,0.587918,35.0,1.0,WFRC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0.0,0,0,0,0,0,COT,1.0506,1.0813,1.0887,1.1071,1.1418,0.9245,0.6061,1.0924,0.7653,0.9245,1.0104,1.0283,1.0243,0.9370,0.9024,0.9471,0.9888,1.0146,1.0278,1.0352,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9.0,1.0434,0.0,"{""paths"": [[[422006.9967, 4481267.654200001], ..."
5837,5837,0,WFRC_8476,WFRC,0.000,0.000,0.589436,35.0,1.0,WFRC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0.0,0,0,0,0,0,COT,1.0506,1.0813,1.0887,1.1071,1.1418,0.9245,0.6061,1.0924,0.7653,0.9245,1.0104,1.0283,1.0243,0.9370,0.9024,0.9471,0.9888,1.0146,1.0278,1.0352,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9.0,1.0434,0.0,"{""paths"": [[[422189.63999999966, 448043

In [298]:
for col in sdf_Forecasts.columns: 
    print(col) 

FID
Id
SEGID
ROUTE
BMP
EMP
DISTANCE
CO_FIPS
SUBAREAID
PLANAREA
F2019
F2023
F2028
F2032
F2042
F2050
CH19TO50
CH19TO23
CH23TO28
CH28TO32
CH32TO42
CH42TO50
AADTSTN
AADT2022
AADT2021
AADT2020
AADT2019
AADT2018
AADT2017
AADT2016
AADT2015
AADT2014
AADT2013
AADT2012
AADT2011
AADT2010
AADT2009
AADT2008
AADT2007
AADT2006
AADT2005
AADT2004
AADT2003
AADT2002
AADT2001
AADT2000
AADT1999
AADT1998
AADT1997
AADT1996
AADT1995
AADT1994
AADT1993
AADT1992
AADT1991
AADT1990
AADT1989
AADT1988
AADT1987
AADT1986
AADT1985
AADT1984
AADT1983
AADT1982
AADT1981
SUTRK2022
CUTRK2022
SUTRK2021
CUTRK2021
SUTRK2020
CUTRK2020
SUTRK2019
CUTRK2019
SUTRK2018
CUTRK2018
SUTRK2017
CUTRK2017
SUTRK2016
CUTRK2016
SUTRK2015
CUTRK2015
SUTRK2014
CUTRK2014
SUTRK2013
CUTRK2013
SUTRK2012
CUTRK2012
SUTRK2011
CUTRK2011
SUTRK2010
CUTRK2010
CCSGROUP19
FAC_MON
FAC_TUE
FAC_WED
FAC_THU
FAC_FRI
FAC_SAT
FAC_SUN
FAC_WDAVG
FAC_WEAVG
FAC_WEMAX
FAC_SPR
FAC_SUM
FAC_FAL
FAC_WIN
FAC_JAN
FAC_FEB
FAC_MAR
FAC_APR
FAC_MAY
FAC_JUN
FAC_JUL
FAC_AUG
FAC_SEP


In [299]:
df = sdf_Forecasts

df = df[(df[nameSegID]=='MAG_6242')]

df

,FID,Id,SEGID,ROUTE,BMP,EMP,DISTANCE,CO_FIPS,SUBAREAID,PLANAREA,F2019,F2023,F2028,F2032,F2042,F2050,CH19TO50,CH19TO23,CH23TO28,CH28TO32,CH32TO42,CH42TO50,AADTSTN,AADT2022,AADT2021,AADT2020,AADT2019,AADT2018,AADT2017,AADT2016,AADT2015,AADT2014,AADT2013,AADT2012,AADT2011,AADT2010,AADT2009,AADT2008,AADT2007,AADT2006,AADT2005,AADT2004,AADT2003,AADT2002,AADT2001,AADT2000,AADT1999,AADT1998,AADT1997,AADT1996,AADT1995,AADT1994,AADT1993,AADT1992,AADT1991,AADT1990,AADT1989,AADT1988,AADT1987,AADT1986,AADT1985,AADT1984,AADT1983,AADT1982,AADT1981,SUTRK2022,CUTRK2022,SUTRK2021,CUTRK2021,SUTRK2020,CUTRK2020,SUTRK2019,CUTRK2019,SUTRK2018,CUTRK2018,SUTRK2017,CUTRK2017,SUTRK2016,CUTRK2016,SUTRK2015,CUTRK2015,SUTRK2014,CUTRK2014,SUTRK2013,CUTRK2013,SUTRK2012,CUTRK2012,SUTRK2011,CUTRK2011,SUTRK2010,CUTRK2010,CCSGROUP19,FAC_MON,FAC_TUE,FAC_WED,FAC_THU,FAC_FRI,FAC_SAT,FAC_SUN,FAC_WDAVG,FAC_WEAVG,FAC_WEMAX,FAC_SPR,FAC_SUM,FAC_FAL,FAC_WIN,FAC_JAN,FAC_FEB,FAC_MAR,FAC_APR,FAC_MAY,FAC_JUN,FAC_JUL,FAC_AUG,FAC_SEP,FAC_OCT,FAC_NOV,FAC_DEC,FAC_MAXMO,FAC_MAX,FACMANADJ,SHAPE
4739,4739,0,MAG_6242,MAG,0.0,0.0,7.437925,49.0,1.0,MAG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0.0,0,0,0,0,0,XX3,1.1071,1.1465,1.1587,1.1651,1.1603,0.7332,0.5291,1.1568,0.6311,0.7332,1.0112,1.0254,1.0105,0.9529,0.9362,0.964,0.9953,1.0139,1.0245,1.0357,1.0085,1.032,1.0208,1.0204,0.9902,0.9587,8.0,1.0414,0.0,"{""paths"": [[[426287.4000000004, 4457858.58], [..."


In [300]:
#normalize data with forecasts

meltIndex    = ['SEGID', 'FAC_WDAVG']
valueColumns = ['AADT2019','F2023','F2028','F2032','F2042','F2050']
df_Forecasts = pd.melt(sdf_Forecasts, id_vars=meltIndex, value_vars=valueColumns)
df_Forecasts.columns = ('SEGID','FAC_WDAVG','FCOL','ForecastAADT')
df_Forecasts['FYEAR'] = (df_Forecasts['FCOL'].str[-4:]).astype('int64')
df_Forecasts['ForecastAADTAdj'] = 0 #placeholder column to preserve location
df_Forecasts['AvgAnnualPrdVol'] = 0 #placeholder column to preserve location
df_Forecasts = pd.merge(df_Forecasts,df_SegmentsMatchedToFactorGroups, on='SEGID')
df_Forecasts = df_Forecasts[['SEGID','FAC_WDAVG','FYEAR','ForecastAADT','ForecastAADTAdj','AvgAnnualPrdVol','FACGEO']]
df_Forecasts

,SEGID,FAC_WDAVG,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO
0,0006_141.0,0.9840,2019,1517.0,0,0,Statewide
1,0006_141.0,0.9840,2023,1600.0,0,0,Statewide
...,...,...,...,...,...,...,...
32092,WFRC_8476,1.0924,2042,0.0,0,0,Statewide
32093,WFRC_8476,1.0924,2050,0.0,0,0,Statewide


In [301]:
df_Forecasts[(df_Forecasts[nameSegID]=='MAG_6242')]

,SEGID,FAC_WDAVG,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO
25500,MAG_6242,1.1568,2019,0.0,0,0,Statewide
25501,MAG_6242,1.1568,2023,0.0,0,0,Statewide
...,...,...,...,...,...,...,...
25504,MAG_6242,1.1568,2042,0.0,0,0,Statewide
25505,MAG_6242,1.1568,2050,0.0,0,0,Statewide


In [302]:
#Hrs within Prd, for capacity
dRoundForecast = {
     'RoundIndex'  : [ 1,  2,   3,    4,     5],
     'ForecastBin' : [ 0,100,1000,10000,100000],
     'RoundToValue': [10, 50, 100,  500,  1000]
}
df_RoundForecast = pd.DataFrame(dRoundForecast)
display(df_RoundForecast)

lRoundBins = df_RoundForecast['ForecastBin'].to_list()
lRoundBase = df_RoundForecast['RoundToValue'].to_list()

def forecast_round(x):
    base = lRoundBase[np.digitize([x],lRoundBins)[0]-1]
    return int(base * round(float(x)/base))

#df = pd.Series([11,16,21]).apply(lambda x: custom_round(x, base=5))

,RoundIndex,ForecastBin,RoundToValue
0,1,0,10
1,2,100,50
...,...,...,...
3,4,10000,500
4,5,100000,1000


In [303]:
#display(lRoundBase[np.digitize([1],lRoundBins)[0]-1])
#display(lRoundBase[np.digitize([54],lRoundBins)[0]-1])
#display(lRoundBase[np.digitize([123],lRoundBins)[0]-1])
#display(lRoundBase[np.digitize([1254],lRoundBins)[0]-1])
#display(lRoundBase[np.digitize([15456],lRoundBins)[0]-1])
#display(lRoundBase[np.digitize([2897651],lRoundBins)[0]-1])
#display(lRoundBase[np.digitize([1564654654654],lRoundBins)[0]-1])
#
#display(forecast_round(1))
#display(forecast_round(54))
#display(forecast_round(123))
#display(forecast_round(1254))
#display(forecast_round(15456))
#display(forecast_round(2897651))
#display(forecast_round(1564654654654))

In [304]:
df_Scenarios_toProcess[~df_Scenarios_toProcess['AdjScenarioCode'].isna()]

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
6,28NB32,2032 SE on a 2028 Network,2032,NoBuild28,v901_NoBuild_SE32_Net28_Summary_SEGID_Detailed...,Yes,TIP28,RTP32
7,28NB42,2042 SE on a 2028 Network,2042,NoBuild28,v901_NoBuild_SE42_Net28_Summary_SEGID_Detailed...,Yes,28NB32,RTP42
...,...,...,...,...,...,...,...,...
14,UF50,2050 Needs-Based,2050,Needs,v901_Needs_SE50_Net50_Summary_SEGID_Detailed.csv,Yes,UF42,RTP50
15,UF50_MAG,2050 Needs-Based - MAG,2050,NeedsMAG,v901_Needs_MAG_SE50_Net50_Summary_SEGID_Detail...,Yes,UF50,RTP50


In [305]:
#calculate forecast DY volume for each scenario to process

dfs_PrdForecasts = []
df_Forecasts_SYEAR_tocsv = pd.DataFrame()

for (idxP, row) in df_Scenarios_toProcess.iterrows():

    _scenarioID = idxP
    
    print(row.loc[nameS] + ' Processing')
    print(idxP)
    #df_base = dfs_TDM_Total[idFromCode(row.loc['AdjScenarioCode'])]

    #display(dfs_TDM[idxP])
    
    df_TDM = dfs_TDM[idxP]
    
    #get TDM year
    _scenarioyear = yearFromID(_scenarioID)
    
    #filter only for year of forecast
    
    df_Forecasts['ForecastAADTAdj'] = df_Forecasts['ForecastAADT'] # set adjusted volume so value is set for unadjusted scenarios
    
    df_Forecasts_SYEAR = df_Forecasts[df_Forecasts.FYEAR == _scenarioyear]
    
    #adjust forecast
    
    for (idxA, row) in df_Scenarios_toAdjust.iterrows():
        if (_scenarioID == idxA):
            #display("adjust this")
            
            #get adjust id
            _adjustId = df_Scenarios_toAdjust.index.get_loc(idxA)

            #display(" Adjust ID: ")
            #display(_adjustId)
            #display(dfs_ForecastAdj[_adjustId])
            
            df_Forecasts_Adjust = pd.DataFrame.merge(df_Forecasts_SYEAR,dfs_ForecastAdj[_adjustId],on='SEGID')


            if idxA==14:
                display(df_Forecasts_Adjust[df_Forecasts_Adjust['SEGID']=='MAG_6242'])

            #adjust volume. if scenario vol = 0 then zero out forecast volume, otherwise forecast volume is adjusted
            df_Forecasts_Adjust.loc[(df_Forecasts_Adjust['DYVolScenario'] == 0), 'ForecastAADTAdj'] = 0
            df_Forecasts_Adjust.loc[(df_Forecasts_Adjust['DYVolScenario']  > 0), 'ForecastAADTAdj'] = df_Forecasts_Adjust['ForecastAADT'] * (1 + df_Forecasts_Adjust['DYVolAdj'])
            
            df_Forecasts_Adjust.loc[(df_Forecasts_Adjust['DYVolScenario'] == 0), 'ForecastAADTAdj'] = 0

            if idxA==14:
                display(df_Forecasts_Adjust[df_Forecasts_Adjust['SEGID']=='MAG_6242'])

            #if adjustment < 0 then bring to 0
            df_Forecasts_Adjust.loc[(df_Forecasts_Adjust['ForecastAADTAdj']< 0), 'ForecastAADTAdj'] = 0  
            
            df_Forecasts_Adjust.loc[((df_Forecasts_Adjust['ForecastAADTAdj']==0) & (df_Forecasts_Adjust['DYVolScenario']>0)), 'ForecastAADTAdj'] = df_Forecasts_Adjust['DYVolScenario'] / df_Forecasts_Adjust['FAC_WDAVG']

            if idxA==14:
                display(df_Forecasts_Adjust[df_Forecasts_Adjust['SEGID']=='MAG_6242'])

            if idxA==14:
                display(df_Forecasts_Adjust[df_Forecasts_Adjust['SEGID']=='MAG_6242']['DYVolAdj'])

            #display(df_Forecasts_Adjust) 
            
            df_Forecasts_Adjust = df_Forecasts_Adjust[['SEGID','FYEAR','ForecastAADT','ForecastAADTAdj','AvgAnnualPrdVol','FACGEO']]
            #df_Forecasts_Adjust = df_Forecasts_Adjust.rename(columns={"DYVolBase": "ForecastAADT"})
            
            #display(df_Forecasts_Adjust['ForecastAADTAdj'])
            df_Forecasts_Adjust['RoundBin'] = np.digitize(df_Forecasts_Adjust['ForecastAADTAdj'],lRoundBins)
            df_Forecasts_Adjust_forRounding = pd.DataFrame.merge(df_Forecasts_Adjust,df_RoundForecast,left_on='RoundBin',right_on='RoundIndex')
            df_Forecasts_Adjust_forRounding['ForecastAADTAdjRounded'] = round((df_Forecasts_Adjust_forRounding['ForecastAADTAdj']/df_Forecasts_Adjust_forRounding['RoundToValue']),0)*df_Forecasts_Adjust_forRounding['RoundToValue']
            df_Forecasts_Adjust_forRounding = df_Forecasts_Adjust_forRounding.fillna(0)
            df_Forecasts_Adjust_forRounding['ForecastAADTAdjRounded'] = df_Forecasts_Adjust_forRounding['ForecastAADTAdjRounded'].astype(int)
            #display(df_Forecasts_Adjust_forRounding[['ForecastAADTAdj','RoundBin','RoundToValue','ForecastAADTAdjRounded']])
            #display(df_Forecasts_Adjust_forRounding[df_Forecasts_Adjust_forRounding['SEGID']=='0091_002.0'])
            
            df_Forecasts_Adjust_forRounding['ForecastAADTAdj'] = df_Forecasts_Adjust_forRounding['ForecastAADTAdjRounded']
            
            df_Forecasts_Adjust_forRounding = df_Forecasts_Adjust_forRounding.drop(columns=['RoundIndex','ForecastBin','RoundBin','RoundToValue','ForecastAADTAdjRounded'])
            
            df_Forecasts_SYEAR = df_Forecasts_Adjust_forRounding.copy()      
            #display(df_Forecasts_SYEAR[df_Forecasts_SYEAR['SEGID']=='0091_002.0'])
            
    df_Forecasts_SYEAR = df_Forecasts_SYEAR.fillna(0)
    #df_Forecasts_SYEAR['ForecastAADT'] = df_Forecasts_SYEAR['ForecastAADT'].astype(int)
    #df_Forecasts_SYEAR['ForecastAADTAdj'] = df_Forecasts_SYEAR['ForecastAADTAdj'].astype(int)
    
    #display(df_Forecasts_SYEAR)

    df_Forecasts_SYEAR_export = df_Forecasts_SYEAR.copy()
    df_Forecasts_SYEAR_export['ScenarioCode'] = codeFromID(_scenarioID)
    df_Forecasts_SYEAR_export['ForecastAADT'] = df_Forecasts_SYEAR_export['ForecastAADTAdj'] #override for export
    df_Forecasts_SYEAR_export = df_Forecasts_SYEAR_export[['ScenarioCode','SEGID','ForecastAADT']]

    #display(df_Forecasts_SYEAR_export[df_Forecasts_SYEAR_export['SEGID']=='0091_002.0'])
    
    df_Forecasts_SYEAR_tocsv = pd.concat([df_Forecasts_SYEAR_tocsv,df_Forecasts_SYEAR_export])
    
    #display(os.path.join(dirResults + "0_Forecast_" + codeFromID(_scenarioID) + ".csv"))
    #df_Forecasts_SYEAR_export.to_csv(os.path.join(dirResults + r"\\0_Forecast_" + codeFromID(_scenarioID) + ".csv"),index=False)
    
    #join with TDM data
    df_Forecasts_withPrd = pd.DataFrame.merge(df_TDM, df_Forecasts_SYEAR, on=('SEGID'))
    
    df_Forecasts_withPrd['AvgAnnualPrdVol'] = df_Forecasts_withPrd['ForecastAADTAdj'] * df_Forecasts_withPrd[nameFGFac] * df_Forecasts_withPrd[nameDFac] * df_Forecasts_withPrd[namePrdFac]
    
    #display(df_Forecasts_withPrd.head(20))
    
    dfs_PrdForecasts.append(df_Forecasts_withPrd)

display(os.path.join(dirResults + r"\\0_Forecasts.csv"))
df_Forecasts_SYEAR_tocsv.to_csv(os.path.join(dirResults + r"\\0_Forecasts.csv"),index=False)
display('CSV Exported')

2023 Base Processing
1
2028 TIP Projects Processing
2
2032 Fiscally-Constrained Processing
3
2042 Fiscally-Constrained Processing
4
2032 SE on a 2028 Network Processing
6
2042 SE on a 2028 Network Processing
7
2050 SE on a 2028 Network Processing
8
2042 SE on a 2032 Network Processing
9
2050 SE on a 2032 Network Processing
10
2050 SE on a 2042 Network Processing
11
2032 Needs-Based Processing
12
2042 Needs-Based Processing
13
2050 Needs-Based Processing
14


,SEGID,FAC_WDAVG,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,DYVolScenario,DYVolBase,DYVolAdj
4074,MAG_6242,1.1568,2050,0.0,0.0,0,Statewide,81098.5,NaN,0.0


,SEGID,FAC_WDAVG,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,DYVolScenario,DYVolBase,DYVolAdj
4074,MAG_6242,1.1568,2050,0.0,0.0,0,Statewide,81098.5,NaN,0.0


,SEGID,FAC_WDAVG,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,DYVolScenario,DYVolBase,DYVolAdj
4074,MAG_6242,1.1568,2050,0.0,70105.895574,0,Statewide,81098.5,NaN,0.0


4074    0.0
Name: DYVolAdj, dtype: float64

2050 Needs-Based - MAG Processing
15


'e:\\GitHub\\V-over-C-Calculations\\results\\\\0_Forecasts.csv'

'CSV Exported'

In [306]:
#format forecasts as table for joining with other analysis
table = pd.pivot_table(df_Forecasts_SYEAR_tocsv, values='ForecastAADT', index=['SEGID'],
                    columns=['ScenarioCode'], aggfunc=np.sum, fill_value=0)
display(table)

table.to_csv(os.path.join(dirResults + r"\\0_Forecasts_Table.csv"))

ScenarioCode,28NB32,28NB42,28NB50,32NB42,32NB50,42NB50,Base23,RTP32,RTP42,TIP28,UF32,UF42,UF50,UF50_MAG
SEGID,,,,,,,,,,,,,,
0006_141.0,1800,2000,2100,2100,2200,2200,1600,1800,2100,1700,1800,2100,2300,2300
0006_146.9,1800,2000,2100,2100,2200,2200,1600,1800,2100,1800,1800,2100,2300,2300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WFRC_8475,0,0,0,0,0,0,0,0,0,0,0,0,0,0
WFRC_8476,7900,9600,9700,8300,8600,8300,0,0,0,0,6600,9000,13500,13000


In [307]:
dftemp = table.reset_index()
dftemp[dftemp['SEGID']=='MAG_6242']

ScenarioCode,SEGID,28NB32,28NB42,28NB50,32NB42,32NB50,42NB50,Base23,RTP32,RTP42,TIP28,UF32,UF42,UF50,UF50_MAG
4250,MAG_6242,0,0,0,0,0,0,0,0,0,0,0,0,70000,69500


In [308]:
segid ='2878_005.4'
pd.set_option("max_rows", 13)
display(df_Forecasts_SYEAR_tocsv[df_Forecasts_SYEAR_tocsv[('SEGID')]==segid])

,ScenarioCode,SEGID,ForecastAADT
18703,Base23,2878_005.4,39500.0
18704,TIP28,2878_005.4,35000.0
18705,RTP32,2878_005.4,28500.0
18706,RTP42,2878_005.4,31500.0
3709,28NB32,2878_005.4,35500.0
...,...,...,...
3806,42NB50,2878_005.4,30500.0
3782,UF32,2878_005.4,27500.0
3825,UF42,2878_005.4,27000.0
3999,UF50,2878_005.4,25500.0


In [309]:
dfs_PrdForecasts[0]

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FAC_WDAVG,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO
0,0006_141.0,1.0,1663.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,11.8,8.5,0.9840,2023,1600.0,1600.0,960.000000,Statewide
1,0006_141.0,1.0,1663.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5862,AM,0.2123,8.6,8.2,0.9840,2023,1600.0,1600.0,199.120416,Statewide
2,0006_141.0,1.0,1663.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5382,MD,0.3103,13.3,6.5,0.9840,2023,1600.0,1600.0,267.205536,Statewide
3,0006_141.0,1.0,1663.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4943,PM,0.2465,10.9,8.3,0.9840,2023,1600.0,1600.0,194.951920,Statewide
4,0006_141.0,1.0,1663.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5166,EV,0.2309,13.9,11.8,0.9840,2023,1600.0,1600.0,190.852704,Statewide
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44285,WFRC_8476,3.0,6740.4,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,0.6000,DY,1.0000,9.1,3.3,1.0924,2023,0.0,0.0,0.000000,Statewide
44286,WFRC_8476,3.0,6740.4,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,0.2008,AM,0.2009,15.4,4.8,1.0924,2023,0.0,0.0,0.000000,Statewide
44287,WFRC_8476,3.0,6740.4,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,0.4805,MD,0.2732,10.3,3.9,1.0924,2023,0.0,0.0,0.000000,Statewide
44288,WFRC_8476,3.0,6740.4,Arterial,1.0,D2,1.0,3.0,Minor Arterial,894,0.6923,PM,0.3136,7.5,3.1,1.0924,2023,0.0,0.0,0.000000,Statewide


In [310]:
df = dfs_PrdForecasts[0]

pd.set_option('display.max_rows', 10)

df[(df[nameSegID]=='2878_005.4')].head(10)

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FAC_WDAVG,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO
31110,2878_005.4,4.0,56513.9,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.6000,DY,1.0000,8.3,3.6,1.0946,2023,39500.0,39500.0,23700.000000,Statewide
31111,2878_005.4,4.0,56513.9,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.6607,AM,0.1595,2.6,0.9,1.0946,2023,39500.0,39500.0,4162.575175,Statewide
31112,2878_005.4,4.0,56513.9,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.5332,MD,0.3250,7.3,2.3,1.0946,2023,39500.0,39500.0,6844.955000,Statewide
31113,2878_005.4,4.0,56513.9,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.4078,PM,0.1986,13.1,5.4,1.0946,2023,39500.0,39500.0,3199.068660,Statewide
31114,2878_005.4,4.0,56513.9,Arterial,1.0,D1,2.0,2.0,Principal Arterial,883,0.4219,EV,0.3169,11.4,6.1,1.0946,2023,39500.0,39500.0,5281.154345,Statewide
31115,2878_005.4,4.0,56513.9,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.6000,DY,1.0000,7.4,3.4,1.0946,2023,39500.0,39500.0,23700.000000,Statewide
31116,2878_005.4,4.0,56513.9,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.3393,AM,0.1595,16.3,5.1,1.0946,2023,39500.0,39500.0,2137.674825,Statewide
31117,2878_005.4,4.0,56513.9,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.4668,MD,0.3250,7.6,3.5,1.0946,2023,39500.0,39500.0,5992.545000,Statewide
31118,2878_005.4,4.0,56513.9,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.5922,PM,0.1986,1.0,0.3,1.0946,2023,39500.0,39500.0,4645.631340,Statewide
31119,2878_005.4,4.0,56513.9,Arterial,1.0,D2,2.0,2.0,Principal Arterial,883,0.5781,EV,0.3169,8.7,4.8,1.0946,2023,39500.0,39500.0,7236.395655,Statewide


# Adjust for Season, DOW

In [311]:
display(df_VCGroup_Seasons)
display(df_VCGroup_DOWPkHr)
display(df_VCGroup_DOWPkHr[['DOWGROUP','PERIOD']].drop_duplicates())

,SeasonCode,SeasonName,SEASONGROUP,SeasonType
0,Ann,Annual,S00-Ann,Season


,DOWPkCode,DOWPkName,DOWGROUP,PERIOD,HRPCTOF
0,WkAM,AM - Weekday,1-Weekday (Tu-Th),AM,Prd
1,WkMD,Midday - Weekday,1-Weekday (Tu-Th),MD,Prd
2,WkPM,PM - Weekday,1-Weekday (Tu-Th),PM,Prd


,DOWGROUP,PERIOD
0,1-Weekday (Tu-Th),AM
1,1-Weekday (Tu-Th),MD
2,1-Weekday (Tu-Th),PM


In [312]:
#read in data from other notebook pre-processing
df_SeasonFactors = pd.read_csv(csv_SeasonFactors)
df_DOWFactors = pd.read_csv(csv_DOWFactors)
df_TODFactors = pd.read_csv(csv_TODFactors)

df_TODFactors.loc[(df_TODFactors['SEASONGROUP']=='Year'),'SEASONGROUP'] = 'S00-Ann'

df_SeasonFactors = pd.DataFrame.merge(df_SeasonFactors, df_VCGroup_Seasons['SEASONGROUP'].drop_duplicates(), on='SEASONGROUP')
df_DOWFactors = pd.DataFrame.merge(df_DOWFactors, df_VCGroup_DOWPkHr['DOWGROUP'].drop_duplicates(), on='DOWGROUP')
df_TODFactors = pd.DataFrame.merge(df_TODFactors, df_VCGroup_Seasons['SEASONGROUP'].drop_duplicates(), on='SEASONGROUP')
df_TODFactors = pd.DataFrame.merge(df_TODFactors, df_VCGroup_DOWPkHr[['DOWGROUP','PERIOD']].drop_duplicates(), on=('DOWGROUP','PERIOD'))

#hard coded for now
df_TODFactors = df_TODFactors[df_TODFactors['YEARGROUP']=='2015-2019']

df_SeasonFactors = df_SeasonFactors.rename(columns={"FuncGroup": nameFG, "AreaType": nameAT})
df_DOWFactors = df_DOWFactors.rename(columns={"FuncGroup": nameFG, "AreaType": nameAT})
df_TODFactors = df_TODFactors.rename(columns={"FuncGroup": nameFG, "AreaType": nameAT})

display(df_SeasonFactors)
display(df_DOWFactors)
display(df_TODFactors)
display(df_TODFactors[df_TODFactors['SEASONGROUP']=='S00-Ann'])

c:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,STATIONGROUP,FACTOR,SEASONGROUP
0,CO0,1.0,S00-Ann
1,CO1,1.0,S00-Ann
2,CO2,1.0,S00-Ann
3,CO3,1.0,S00-Ann
4,CO4,1.0,S00-Ann
...,...,...,...
32,COW,1.0,S00-Ann
33,COX,1.0,S00-Ann
34,COY,1.0,S00-Ann
35,COZ,1.0,S00-Ann


,STATIONGROUP,FACTOR,DOWGROUP
0,CO0,0.715249,1-Weekday (Tu-Th)
1,CO1,0.824043,1-Weekday (Tu-Th)
2,CO2,0.790750,1-Weekday (Tu-Th)
3,CO3,0.853478,1-Weekday (Tu-Th)
4,CO4,0.904970,1-Weekday (Tu-Th)
...,...,...,...
32,COW,1.067677,1-Weekday (Tu-Th)
33,COX,1.087913,1-Weekday (Tu-Th)
34,COY,0.967245,1-Weekday (Tu-Th)
35,COZ,1.006499,1-Weekday (Tu-Th)


,STATIONGROUP,YEARGROUP,SEASONGROUP,DOWGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
4,301,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.4054,Prd
12,302,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.3652,Prd
19,303,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.3819,Prd
28,304,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.5003,Prd
37,305,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.4619,Prd
...,...,...,...,...,...,...,...
11161,SNC,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0836,DY
11170,XX1,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY
11179,XX2,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0804,DY
11188,XX3,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY


,STATIONGROUP,YEARGROUP,SEASONGROUP,DOWGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
4,301,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.4054,Prd
12,302,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.3652,Prd
19,303,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.3819,Prd
28,304,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.5003,Prd
37,305,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.4619,Prd
...,...,...,...,...,...,...,...
11161,SNC,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0836,DY
11170,XX1,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY
11179,XX2,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0804,DY
11188,XX3,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY


In [313]:
df_FacGeoFunGroupAreaTypeVolume_ToStationGroup = pd.read_csv(csv_FacGeoFunGroupAreaTypeVolume_ToStationGroup)
display(df_FacGeoFunGroupAreaTypeVolume_ToStationGroup)
display(df_FacGeoFunGroupAreaTypeVolume_ToStationGroup[(df_FacGeoFunGroupAreaTypeVolume_ToStationGroup['FACGEO']=='Statewide') & (df_FacGeoFunGroupAreaTypeVolume_ToStationGroup['AREATYPE']==1) & (df_FacGeoFunGroupAreaTypeVolume_ToStationGroup['FUNCGROUP']=='Arterial')])

,FACGEO,AREATYPE,VOLUMEFROM,VOLUMETO,FUNCGROUP,STATIONGROUP
0,I80NevSR36,1,0,1000000,Arterial,COQ
1,I80NevSR36,2,0,1000000,Arterial,COS
2,I80NevSR36,3,0,1000000,Arterial,COT
3,I80NevSR36,4,0,1000000,Arterial,COU
4,I80NevSR36,5,0,1000000,Arterial,COU
...,...,...,...,...,...,...
583,Southeast,4,1500,1000000,CD Road,XX3
584,Southeast,5,1500,1000000,CD Road,XX3
585,Statewide,1,0,2500,CD Road,XX3
586,Statewide,1,2500,12500,CD Road,XX3


,FACGEO,AREATYPE,VOLUMEFROM,VOLUMETO,FUNCGROUP,STATIONGROUP
144,Statewide,1,0,2500,Arterial,COP
145,Statewide,1,2500,12500,Arterial,COQ
146,Statewide,1,12500,1000000,Arterial,COR


In [314]:
#add season factor

dfs_PrdForecasts_Season = []

#for (idx, row) in dfs_PrdForecasts.iterrows():

for df_PrdForecasts in dfs_PrdForecasts: 
    
    #joing forecasts data with station group matching
    df_PrdForecasts_Season = pd.DataFrame.merge(df_PrdForecasts,df_FacGeoFunGroupAreaTypeVolume_ToStationGroup,on=('FACGEO',nameFG,nameAT),how='left')
    #display(df_PrdForecasts_Season)
    
    #remove unapplicable volume classes
    df_PrdForecasts_Season = df_PrdForecasts_Season[(df_PrdForecasts_Season.ForecastAADT >= df_PrdForecasts_Season.VOLUMEFROM) & (df_PrdForecasts_Season.ForecastAADT < df_PrdForecasts_Season.VOLUMETO)]

    #join to season factors
    df_PrdForecasts_Season = pd.DataFrame.merge(df_PrdForecasts_Season,df_SeasonFactors,on=('STATIONGROUP'),how='left')
    
    #calculate Average Season
    df_PrdForecasts_Season['AvgSsnPrdVol'] = df_PrdForecasts_Season['AvgAnnualPrdVol'] * df_PrdForecasts_Season['FACTOR'] 
    df_PrdForecasts_Season['AvgSsnPrdVol'] = df_PrdForecasts_Season['AvgSsnPrdVol'].astype(int)
    
    
    df_PrdForecasts_Season = df_PrdForecasts_Season.rename(columns=({'FACTOR':'SsnFactor'}))
    
    #display(df_PrdForecasts_Season)
    
    dfs_PrdForecasts_Season.append(df_PrdForecasts_Season)
    

In [315]:
#dfs_PrdForecasts[6]
#segid='2878_005.4'
#i=4
#display(dfs_PrdForecasts[i][dfs_PrdForecasts[i]['SEGID']==segid])
#display(df_SeasonFactors[df_SeasonFactors['STATIONGROUP']=='COT'])
#display(dfs_PrdForecasts_Season[i][dfs_PrdForecasts_Season[i]['SEGID']==segid])

In [316]:
df_DOWFactors

,STATIONGROUP,FACTOR,DOWGROUP
0,CO0,0.715249,1-Weekday (Tu-Th)
1,CO1,0.824043,1-Weekday (Tu-Th)
2,CO2,0.790750,1-Weekday (Tu-Th)
3,CO3,0.853478,1-Weekday (Tu-Th)
4,CO4,0.904970,1-Weekday (Tu-Th)
...,...,...,...
32,COW,1.067677,1-Weekday (Tu-Th)
33,COX,1.087913,1-Weekday (Tu-Th)
34,COY,0.967245,1-Weekday (Tu-Th)
35,COZ,1.006499,1-Weekday (Tu-Th)


In [317]:
#add dow factor

dfs_PrdForecasts_Season_DOW = []

#for (idx, row) in dfs_PrdForecasts.iterrows():

for df_PrdForecasts_Season in dfs_PrdForecasts_Season: 
    
    #joing forecasts data with season factors
    df_PrdForecasts_Season_DOW = pd.DataFrame.merge(df_PrdForecasts_Season,df_DOWFactors,on=('STATIONGROUP'),how='left')
    
    #calculate Average Season DOW
    df_PrdForecasts_Season_DOW['AvgSsnDOWPrdVol'] = df_PrdForecasts_Season_DOW['AvgSsnPrdVol'] * df_PrdForecasts_Season_DOW['FACTOR'] 
    df_PrdForecasts_Season_DOW['AvgSsnDOWPrdVol'] = df_PrdForecasts_Season_DOW['AvgSsnDOWPrdVol'].astype(int)

    df_PrdForecasts_Season_DOW = df_PrdForecasts_Season_DOW.rename(columns=({'FACTOR':'DOWFactor'}))
    
    #display(df_PrdForecasts_Season_DOW)
    
    dfs_PrdForecasts_Season_DOW.append(df_PrdForecasts_Season_DOW)

In [318]:
df_TODFactors

,STATIONGROUP,YEARGROUP,SEASONGROUP,DOWGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
4,301,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.4054,Prd
12,302,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.3652,Prd
19,303,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.3819,Prd
28,304,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.5003,Prd
37,305,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.4619,Prd
...,...,...,...,...,...,...,...
11161,SNC,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0836,DY
11170,XX1,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY
11179,XX2,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0804,DY
11188,XX3,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY


In [319]:
df_VCGroups

,SEASONGROUP,SeasonType,DOWGROUP,PERIOD,HRPCTOF,VCGroupCode
VCGroupID,,,,,,
0,S00-Ann,Season,1-Weekday (Tu-Th),AM,Prd,AnnWkAM
1,S00-Ann,Season,1-Weekday (Tu-Th),MD,Prd,AnnWkMD
2,S00-Ann,Season,1-Weekday (Tu-Th),PM,Prd,AnnWkPM


In [320]:
df_PrdForecasts_Season_DOW

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol
0,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6000,DY,1.0000,12.2,8.6,2050,2300.0,2300,1380.000000,Statewide,0,2500,COP,1.0,S00-Ann,1380,0.984028,1-Weekday (Tu-Th),1357
1,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6145,AM,0.1973,9.0,7.6,2050,2300.0,2300,278.853955,Statewide,0,2500,COP,1.0,S00-Ann,278,0.984028,1-Weekday (Tu-Th),273
2,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5453,MD,0.3061,13.3,7.5,2050,2300.0,2300,383.907559,Statewide,0,2500,COP,1.0,S00-Ann,383,0.984028,1-Weekday (Tu-Th),376
3,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.4731,PM,0.2642,11.4,8.6,2050,2300.0,2300,287.483946,Statewide,0,2500,COP,1.0,S00-Ann,287,0.984028,1-Weekday (Tu-Th),282
4,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5121,EV,0.2324,14.8,11.4,2050,2300.0,2300,273.727692,Statewide,0,2500,COP,1.0,S00-Ann,273,0.984028,1-Weekday (Tu-Th),268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52665,WFRC_8476,4.0,14314.3,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,0.6000,DY,1.0000,12.2,4.5,2050,0.0,13000,7800.000000,Statewide,0,1000000,COU,1.0,S00-Ann,7800,1.094640,1-Weekday (Tu-Th),8538
52666,WFRC_8476,4.0,14314.3,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,0.2497,AM,0.1854,13.8,5.4,2050,0.0,13000,601.826940,Statewide,0,1000000,COU,1.0,S00-Ann,601,1.094640,1-Weekday (Tu-Th),657
52667,WFRC_8476,4.0,14314.3,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,0.3592,MD,0.2937,13.4,3.2,2050,0.0,13000,1371.461520,Statewide,0,1000000,COU,1.0,S00-Ann,1371,1.094640,1-Weekday (Tu-Th),1500
52668,WFRC_8476,4.0,14314.3,Arterial,1.0,D2,1.0,3.0,Minor Arterial,849,0.5298,PM,0.2887,9.5,5.6,2050,0.0,13000,1988.392380,Statewide,0,1000000,COU,1.0,S00-Ann,1988,1.094640,1-Weekday (Tu-Th),2176


In [321]:
df_PrdForecasts_Season_DOW.columns

Index(['SEGID', 'AREATYPE', 'SegDYVol_TDM', 'FUNCGROUP', 'FGFAC', 'Dir',
       'LANES', 'FT', 'FTCLASS', 'CAP1HL', 'DFAC', 'Prd', 'PRDFAC', 'MDPCT',
       'HVPCT', 'FYEAR', 'ForecastAADT', 'ForecastAADTAdj', 'AvgAnnualPrdVol',
       'FACGEO', 'VOLUMEFROM', 'VOLUMETO', 'STATIONGROUP', 'SsnFactor',
       'SEASONGROUP', 'AvgSsnPrdVol', 'DOWFactor', 'DOWGROUP',
       'AvgSsnDOWPrdVol'],
      dtype='object')

In [322]:
#add tod factor

segid = '2878_005.4'

pd.set_option('display.max_columns', None)

dfs_PrdForecasts_Season_DOW_TOD_wGroups = []

i=0

for df_PrdForecasts_Season_DOW in dfs_PrdForecasts_Season_DOW: 

    display(i)

    df_PrdForecasts_Season_DOW = df_PrdForecasts_Season_DOW.rename(columns={'Prd': namePrd + '_TDM'})

    #display(df_PrdForecasts_Season_DOW[namePrd + '_TDM'])
    
    #joing forecasts data with season factors
    
    df_TODFactors_Prd = df_TODFactors[df_TODFactors['HRPCTOF'] == 'Prd']
    df_TODFactors_DY  = df_TODFactors[df_TODFactors['HRPCTOF'] == 'DY']
    
    #get peak Hr as a Pct of Prd
    df_PrdForecasts_Season_DOW_TOD_Prd = pd.DataFrame.merge(df_PrdForecasts_Season_DOW,df_TODFactors_Prd,left_on=('STATIONGROUP','SEASONGROUP','DOWGROUP',namePrd + '_TDM'),right_on=('STATIONGROUP','SEASONGROUP','DOWGROUP','PERIOD'))
    #display(df_PrdForecasts_Season_DOW_TOD_Prd[df_PrdForecasts_Season_DOW_TOD_Prd['SEGID']==segid])
    
    #get peak Hr as a Pct of DY
    df_PrdForecasts_Season_DOW_TOD_DY = pd.DataFrame.merge(df_PrdForecasts_Season_DOW,df_TODFactors_DY,left_on=('STATIONGROUP','SEASONGROUP','DOWGROUP',namePrd + '_TDM'),right_on=('STATIONGROUP','SEASONGROUP','DOWGROUP','HRPCTOF'))
 
    df_PrdForecasts_Season_DOW_TOD = pd.concat([df_PrdForecasts_Season_DOW_TOD_Prd, df_PrdForecasts_Season_DOW_TOD_DY])

    #display(df_PrdForecasts_Season_DOW_TOD)
    
    df_PrdForecasts_Season_DOW_TOD_wGroups = pd.DataFrame.merge(df_PrdForecasts_Season_DOW_TOD, df_VCGroups,on=('SEASONGROUP','DOWGROUP','HRPCTOF','PERIOD'))
      
    dfs_PrdForecasts_Season_DOW_TOD_wGroups.append(df_PrdForecasts_Season_DOW_TOD_wGroups)
    
    #display(df_PrdForecasts_Season_DOW_TOD)
    
    i=i+1


0

1

2

3

4

...

9

10

11

12

13

In [323]:
#TROUBLESHOOTING

#df_PrdForecasts_Season_DOW_TOD_DY[(df_PrdForecasts_Season_DOW_TOD_DY['SEGID']=='0006_146.9') & (df_PrdForecasts_Season_DOW_TOD_DY['DOWGroup']=='6-Saturday')].head(20)

#segid = '2878_005.4'
#i=4
#
#ccsgroup = 'COT'
#display(dfs_PrdForecasts_Season[i][dfs_PrdForecasts_Season[i]['SEGID']==segid])
#display(dfs_PrdForecasts_Season_DOW[i][dfs_PrdForecasts_Season_DOW[i]['SEGID']==segid])
#display(dfs_PrdForecasts_Season_DOW_TOD_wGroups[i][dfs_PrdForecasts_Season_DOW_TOD_wGroups[i]['SEGID']==segid])
#
#display(df_TODFactors_Prd[df_TODFactors_Prd['STATIONGROUP']==ccsgroup])
#
#display(df_PrdForecasts_Season_DOW_TOD_Prd[df_PrdForecasts_Season_DOW_TOD_Prd['SEGID']==segid])
#
#display(df_PrdForecasts_Season_DOW_TOD_DY[(df_PrdForecasts_Season_DOW_TOD_DY['SEGID']==segid)].head(20))


In [324]:
#df_PrdForecasts_Season_DOW_TOD[df_PrdForecasts_Season_DOW_TOD['SEGID']=='0006_146.9'].head(20)

In [325]:
#dftemp = dfs_PrdForecasts_Season_DOW_TOD_wGroups[0].groupby(['VCGroupCode']).agg({'VolMaxHrPct':[np.size]})
df_TODFactors_DY[df_TODFactors_DY['SEASONGROUP']=='S00-Ann']

,STATIONGROUP,YEARGROUP,SEASONGROUP,DOWGROUP,PERIOD,VOLPKHRPCT,HRPCTOF
1871,301,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.0749,DY
1879,302,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.0670,DY
1886,303,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.0614,DY
1895,304,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.0435,DY
1904,305,2015-2019,S00-Ann,1-Weekday (Tu-Th),AM,0.0575,DY
...,...,...,...,...,...,...,...
11161,SNC,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0836,DY
11170,XX1,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY
11179,XX2,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0804,DY
11188,XX3,2015-2019,S00-Ann,1-Weekday (Tu-Th),PM,0.0985,DY


In [326]:
df_PrdDuration

,PERIOD,PRDHRS
0,AM,3
1,MD,6
2,PM,3
3,EV,12
4,DY,24


In [327]:
dfs_PkHrForecasts = []

for df_PrdForecasts_Season_DOW_TOD_wGroups in dfs_PrdForecasts_Season_DOW_TOD_wGroups: 
    
    df_PkHrForecasts = pd.DataFrame.merge(df_PrdForecasts_Season_DOW_TOD_wGroups,df_PrdDuration,on='PERIOD',how='left')   
    
    df_PkHrForecasts['AvgSsnDOWPkHrVol'] = df_PkHrForecasts['AvgSsnDOWPrdVol'] * df_PkHrForecasts['VOLPKHRPCT']

    #add truck Pctage adjustments
    #PCE for medium and heavy trucks from HCM 2000 Exhibit 23-8
    Emd = 1.2 #same as RVs
    Ehv = 1.5
    df_PkHrForecasts['TrkFac'] = 1 + (df_PkHrForecasts[nameMDPct]/100 * (Emd-1)) + (df_PkHrForecasts[nameHVPct]/100 * (Ehv-1))
    
    #calculate PCE
    df_PkHrForecasts['PrdPCEFlow' ] = (df_PkHrForecasts['AvgSsnDOWPrdVol' ] *  df_PkHrForecasts['TrkFac']) /  df_PkHrForecasts['PRDHRS']
    df_PkHrForecasts['PkHrPCEFlow'] =  df_PkHrForecasts['AvgSsnDOWPkHrVol'] *  df_PkHrForecasts['TrkFac']
    df_PkHrForecasts['PrdVC'      ] =  df_PkHrForecasts['PrdPCEFlow'      ] / (df_PkHrForecasts[nameCap1HL] * df_PkHrForecasts[nameLanes])
    df_PkHrForecasts['PkHrVC'     ] =  df_PkHrForecasts['PkHrPCEFlow'     ] / (df_PkHrForecasts[nameCap1HL] * df_PkHrForecasts[nameLanes])
    
    display(df_PkHrForecasts)
    
    dfs_PkHrForecasts.append(df_PkHrForecasts)

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FAC_WDAVG,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,1663.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5862,AM,0.2123,8.6,8.2,0.9840,2023,1600.0,1600.0,199.120416,Statewide,0,2500,COP,1.0,S00-Ann,199,0.984028,1-Weekday (Tu-Th),195,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,79.8135,1.0582,68.783000,84.458646,0.064585,0.079304
1,0006_141.0,1.0,1663.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.4138,AM,0.2123,6.3,9.6,0.9840,2023,1600.0,1600.0,140.559584,Statewide,0,2500,COP,1.0,S00-Ann,140,0.984028,1-Weekday (Tu-Th),137,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,56.0741,1.0606,48.434067,59.472190,0.045478,0.055842
2,0006_146.9,1.0,1671.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6479,AM,0.2124,9.1,7.8,0.9840,2023,1600.0,1600.0,220.182336,Statewide,0,2500,COP,1.0,S00-Ann,220,0.984028,1-Weekday (Tu-Th),216,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,88.4088,1.0572,76.118400,93.465783,0.071473,0.087761
3,0006_146.9,1.0,1671.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3521,AM,0.2124,4.8,10.4,0.9840,2023,1600.0,1600.0,119.657664,Statewide,0,2500,COP,1.0,S00-Ann,119,0.984028,1-Weekday (Tu-Th),117,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,47.8881,1.0616,41.402400,50.838007,0.038875,0.047735
4,0065_000.1,1.0,399.0,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6750,AM,0.2005,7.4,1.9,0.9840,2023,1600.0,1600.0,216.540000,Statewide,0,2500,COP,1.0,S00-Ann,216,0.984028,1-Weekday (Tu-Th),212,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,86.7716,1.0243,72.383867,88.880150,0.089806,0.110273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26563,0215_025.7,3.0,121975.2,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4446,PM,0.2705,10.2,6.2,1.1349,2023,85000.0,85000.0,10222.465500,I215,0,1000000,CO8,1.0,S00-Ann,10222,1.134873,1-Weekday (Tu-Th),11600,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4245.6000,1.0514,4065.413333,4463.823840,0.543214,0.596449
26564,0215_026.7,3.0,58801.3,Freeway,1.0,D1,3.0,34.0,Freeway,1964,0.5220,PM,0.2452,19.8,11.9,1.1349,2023,27000.0,27000.0,3455.848800,I215,0,1000000,CO8,1.0,S00-Ann,3455,1.134873,1-Weekday (Tu-Th),3920,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1434.7200,1.0991,1436.157333,1576.900752,0.243747,0.267634
26565,0215_026.7,3.0,58801.3,Freeway,1.0,D2,2.0,34.0,Freeway,2080,0.4780,PM,0.2452,21.6,13.2,1.1349,2023,27000.0,27000.0,3164.551200,I215,0,1000000,CO8,1.0,S00-Ann,3164,1.134873,1-Weekday (Tu-Th),3590,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1313.9400,1.1092,1327.342667,1457.422248,0.319073,0.350342
26566,0215_027.4,3.0,39438.0,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.5606,PM,0.2461,20.6,11.7,1.1349,2023,23500.0,23500.0,3242.146010,I215,0,1000000,CO8,1.0,S00-Ann,3242,1.134873,1-Weekday (Tu-Th),3679,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1346.5140,1.0997,1348.598767,1480.761446,0.324182,0.355952


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FAC_WDAVG,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,1781.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5827,AM,0.2106,9.1,7.5,0.9840,2028,1700.0,1700.0,208.618254,Statewide,0,2500,COP,1.0,S00-Ann,208,0.984028,1-Weekday (Tu-Th),204,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,83.4972,1.0557,71.787600,88.147994,0.067406,0.082768
1,0006_141.0,1.0,1781.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.4173,AM,0.2106,7.1,7.5,0.9840,2028,1700.0,1700.0,149.401746,Statewide,0,2500,COP,1.0,S00-Ann,149,0.984028,1-Weekday (Tu-Th),146,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,59.7578,1.0517,51.182733,62.847278,0.048059,0.059012
2,0006_146.9,1.0,1786.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6419,AM,0.2111,9.5,7.4,0.9840,2028,1800.0,1800.0,243.909162,Statewide,0,2500,COP,1.0,S00-Ann,243,0.984028,1-Weekday (Tu-Th),239,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,97.8227,1.0560,84.128000,103.300771,0.078993,0.096996
3,0006_146.9,1.0,1786.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3581,AM,0.2111,5.9,7.4,0.9840,2028,1800.0,1800.0,136.070838,Statewide,0,2500,COP,1.0,S00-Ann,136,0.984028,1-Weekday (Tu-Th),133,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,54.4369,1.0488,46.496800,57.093421,0.043659,0.053609
4,0065_000.1,1.0,431.0,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6747,AM,0.1926,5.4,0.0,0.9840,2028,1600.0,1600.0,207.915552,Statewide,0,2500,COP,1.0,S00-Ann,207,0.984028,1-Weekday (Tu-Th),203,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,83.0879,1.0108,68.397467,83.985249,0.084860,0.104200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27037,0215_025.7,3.0,129195.0,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4369,PM,0.2590,11.6,7.3,1.1349,2028,91500.0,91500.0,10353.874650,I215,0,1000000,CO8,1.0,S00-Ann,10353,1.134873,1-Weekday (Tu-Th),11749,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4300.1340,1.0597,4150.138433,4556.852000,0.554535,0.608879
27038,0215_026.7,3.0,56537.5,Freeway,1.0,D1,3.0,34.0,Freeway,1964,0.5606,PM,0.2628,11.7,7.0,1.1349,2028,25000.0,25000.0,3683.142000,I215,0,1000000,CO8,1.0,S00-Ann,3683,1.134873,1-Weekday (Tu-Th),4179,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1529.5140,1.0584,1474.351200,1618.837618,0.250229,0.274752
27039,0215_026.7,3.0,56537.5,Freeway,1.0,D2,2.0,34.0,Freeway,2080,0.4394,PM,0.2628,14.3,8.9,1.1349,2028,25000.0,25000.0,2886.858000,I215,0,1000000,CO8,1.0,S00-Ann,2886,1.134873,1-Weekday (Tu-Th),3275,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1198.6500,1.0731,1171.467500,1286.271315,0.281603,0.309200
27040,0215_027.4,3.0,37297.2,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.6149,PM,0.2756,8.9,5.2,1.1349,2028,21500.0,21500.0,3643.528460,I215,0,1000000,CO8,1.0,S00-Ann,3643,1.134873,1-Weekday (Tu-Th),4134,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1513.0440,1.0438,1438.356400,1579.315327,0.345759,0.379643


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FAC_WDAVG,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,1871.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6022,AM,0.2031,8.7,6.6,0.9840,2032,1800.0,1800.0,220.152276,Statewide,0,2500,COP,1.0,S00-Ann,220,0.984028,1-Weekday (Tu-Th),216,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,88.4088,1.0504,75.628800,92.864604,0.071013,0.087197
1,0006_141.0,1.0,1871.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3978,AM,0.2031,5.9,6.5,0.9840,2032,1800.0,1800.0,145.427724,Statewide,0,2500,COP,1.0,S00-Ann,145,0.984028,1-Weekday (Tu-Th),142,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,58.1206,1.0443,49.430200,60.695343,0.046413,0.056991
2,0006_146.9,1.0,1882.8,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6754,AM,0.2029,9.3,6.6,0.9840,2032,1800.0,1800.0,246.669588,Statewide,0,2500,COP,1.0,S00-Ann,246,0.984028,1-Weekday (Tu-Th),242,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,99.0506,1.0516,84.829067,104.161611,0.079652,0.097804
3,0006_146.9,1.0,1882.8,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3246,AM,0.2029,4.0,6.5,0.9840,2032,1800.0,1800.0,118.550412,Statewide,0,2500,COP,1.0,S00-Ann,118,0.984028,1-Weekday (Tu-Th),116,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,47.4788,1.0405,40.232667,49.401691,0.037777,0.046387
4,0065_000.1,1.0,453.8,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6019,AM,0.1882,7.8,1.9,0.9840,2032,1600.0,1600.0,181.244128,Statewide,0,2500,COP,1.0,S00-Ann,181,0.984028,1-Weekday (Tu-Th),178,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,72.8554,1.0251,60.822600,74.684071,0.075462,0.092660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28309,0215_025.7,3.0,144038.4,Freeway,1.0,D2,5.0,34.0,Freeway,1784,0.4412,PM,0.2527,12.4,7.1,1.1349,2032,101000.0,101000.0,11260.615240,I215,0,1000000,CO8,1.0,S00-Ann,11260,1.134873,1-Weekday (Tu-Th),12778,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4676.7480,1.0603,4516.171133,4958.755904,0.506297,0.555914
28310,0215_026.7,3.0,61961.0,Freeway,1.0,D1,4.0,34.0,Freeway,1871,0.5411,PM,0.2497,19.9,10.2,1.1349,2032,27500.0,27500.0,3715.598425,I215,0,1000000,CO8,1.0,S00-Ann,3715,1.134873,1-Weekday (Tu-Th),4216,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1543.0560,1.0908,1532.937600,1683.165485,0.204829,0.224902
28311,0215_026.7,3.0,61961.0,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4589,PM,0.2497,13.6,7.0,1.1349,2032,27500.0,27500.0,3151.151575,I215,0,1000000,CO8,1.0,S00-Ann,3151,1.134873,1-Weekday (Tu-Th),3575,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1308.4500,1.0622,1265.788333,1389.835590,0.169133,0.185708
28312,0215_027.4,3.0,50607.5,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.5447,PM,0.2003,23.2,12.2,1.1349,2032,33000.0,33000.0,3600.412530,I215,0,1000000,CO8,1.0,S00-Ann,3600,1.134873,1-Weekday (Tu-Th),4085,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1495.1100,1.1074,1507.909667,1655.684814,0.362478,0.398001


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FAC_WDAVG,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,2115.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6118,AM,0.2047,8.4,7.5,0.9840,2042,2100.0,2100.0,262.994466,Statewide,0,2500,COP,1.0,S00-Ann,262,0.984028,1-Weekday (Tu-Th),257,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,105.1901,1.0543,90.318367,110.901922,0.084806,0.104133
1,0006_141.0,1.0,2115.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3882,AM,0.2047,5.3,8.4,0.9840,2042,2100.0,2100.0,166.875534,Statewide,0,2500,COP,1.0,S00-Ann,166,0.984028,1-Weekday (Tu-Th),163,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,66.7159,1.0526,57.191267,70.225156,0.053701,0.065939
2,0006_146.9,1.0,2122.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6920,AM,0.2050,9.0,7.3,0.9840,2042,2100.0,2100.0,297.906000,Statewide,0,2500,COP,1.0,S00-Ann,297,0.984028,1-Weekday (Tu-Th),292,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,119.5156,1.0545,102.638000,126.029200,0.096374,0.118337
3,0006_146.9,1.0,2122.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3080,AM,0.2050,3.0,9.0,0.9840,2042,2100.0,2100.0,132.594000,Statewide,0,2500,COP,1.0,S00-Ann,132,0.984028,1-Weekday (Tu-Th),129,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,52.7997,1.0510,45.193000,55.492485,0.042435,0.052106
4,0065_000.1,1.0,537.8,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6514,AM,0.1867,3.1,1.5,0.9840,2042,1700.0,1700.0,206.747846,Statewide,0,2500,COP,1.0,S00-Ann,206,0.984028,1-Weekday (Tu-Th),202,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,82.6786,1.0137,68.255800,83.811297,0.084685,0.103984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29053,0215_025.7,3.0,158610.4,Freeway,1.0,D2,5.0,34.0,Freeway,1784,0.4349,PM,0.2482,11.9,6.5,1.1349,2042,105000.0,105000.0,11333.928900,I215,0,1000000,CO8,1.0,S00-Ann,11333,1.134873,1-Weekday (Tu-Th),12861,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4707.1260,1.0563,4528.358100,4972.137194,0.507663,0.557414
29054,0215_026.7,3.0,66703.6,Freeway,1.0,D1,4.0,34.0,Freeway,1871,0.5625,PM,0.2689,16.3,8.6,1.1349,2042,28500.0,28500.0,4310.803125,I215,0,1000000,CO8,1.0,S00-Ann,4310,1.134873,1-Weekday (Tu-Th),4891,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1790.1060,1.0756,1753.586533,1925.438014,0.234311,0.257274
29055,0215_026.7,3.0,66703.6,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4375,PM,0.2689,12.6,6.6,1.1349,2042,28500.0,28500.0,3352.846875,I215,0,1000000,CO8,1.0,S00-Ann,3352,1.134873,1-Weekday (Tu-Th),3804,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1392.2640,1.0582,1341.797600,1473.293765,0.179289,0.196859
29056,0215_027.4,3.0,54625.5,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.5597,PM,0.2271,18.5,12.4,1.1349,2042,35000.0,35000.0,4448.775450,I215,0,1000000,CO8,1.0,S00-Ann,4448,1.134873,1-Weekday (Tu-Th),5047,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1847.2020,1.0990,1848.884333,2030.074998,0.444443,0.487999


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,1832.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6098,AM,0.2003,9.3,6.5,2032,1800.0,1800,219.857292,Statewide,0,2500,COP,1.0,S00-Ann,219,0.984028,1-Weekday (Tu-Th),215,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,87.9995,1.0511,75.328833,92.496274,0.070731,0.086851
1,0006_141.0,1.0,1832.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3902,AM,0.2003,8.5,5.3,2032,1800.0,1800,140.682708,Statewide,0,2500,COP,1.0,S00-Ann,140,0.984028,1-Weekday (Tu-Th),137,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,56.0741,1.0435,47.653167,58.513323,0.044745,0.054942
2,0006_146.9,1.0,1840.8,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6883,AM,0.2005,9.4,6.7,2032,1800.0,1800,248.407470,Statewide,0,2500,COP,1.0,S00-Ann,248,0.984028,1-Weekday (Tu-Th),244,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,99.8692,1.0523,85.587067,105.092359,0.080363,0.098678
3,0006_146.9,1.0,1840.8,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3117,AM,0.2005,7.8,4.3,2032,1800.0,1800,112.492530,Statewide,0,2500,COP,1.0,S00-Ann,112,0.984028,1-Weekday (Tu-Th),110,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,45.0230,1.0371,38.027000,46.693353,0.035706,0.043844
4,0065_000.1,1.0,477.4,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6444,AM,0.1885,3.4,1.7,2032,1600.0,1700,206.497980,Statewide,0,2500,COP,1.0,S00-Ann,206,0.984028,1-Weekday (Tu-Th),202,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,82.6786,1.0153,68.363533,83.943583,0.084818,0.104148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27043,0215_025.7,3.0,131624.3,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4333,PM,0.2568,11.4,6.9,2032,101000.0,92500,10292.608200,I215,0,1000000,CO8,1.0,S00-Ann,10292,1.134873,1-Weekday (Tu-Th),11680,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4274.8800,1.0573,4116.421333,4519.830624,0.550030,0.603932
27044,0215_026.7,3.0,57214.9,Freeway,1.0,D1,3.0,34.0,Freeway,1964,0.5679,PM,0.2622,10.9,7.2,2032,27500.0,25500,3797.036190,I215,0,1000000,CO8,1.0,S00-Ann,3797,1.134873,1-Weekday (Tu-Th),4309,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1577.0940,1.0578,1519.353400,1668.250033,0.257867,0.283138
27045,0215_026.7,3.0,57214.9,Freeway,1.0,D2,2.0,34.0,Freeway,2080,0.4321,PM,0.2622,14.4,9.0,2032,27500.0,25500,2889.063810,I215,0,1000000,CO8,1.0,S00-Ann,2889,1.134873,1-Weekday (Tu-Th),3278,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1199.7480,1.0738,1173.305467,1288.289402,0.282045,0.309685
27046,0215_027.4,3.0,37817.7,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.6248,PM,0.2773,8.2,4.8,2032,33000.0,24500,4244.797480,I215,0,1000000,CO8,1.0,S00-Ann,4244,1.134873,1-Weekday (Tu-Th),4816,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1762.6560,1.0404,1670.188800,1833.867302,0.401488,0.440833


...

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,2303.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6067,AM,0.2006,9.1,7.7,2050,2300.0,2200,267.748844,Statewide,0,2500,COP,1.0,S00-Ann,267,0.984028,1-Weekday (Tu-Th),262,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,107.2366,1.0567,92.285133,113.316915,0.086653,0.106401
1,0006_141.0,1.0,2303.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3933,AM,0.2006,7.4,8.4,2050,2300.0,2200,173.571156,Statewide,0,2500,COP,1.0,S00-Ann,173,0.984028,1-Weekday (Tu-Th),170,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,69.5810,1.0568,59.885333,73.533201,0.056230,0.069045
2,0006_146.9,1.0,2307.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6832,AM,0.2011,9.5,7.6,2050,2300.0,2200,302.261344,Statewide,0,2500,COP,1.0,S00-Ann,302,0.984028,1-Weekday (Tu-Th),297,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,121.5621,1.0570,104.643000,128.491140,0.098256,0.120649
3,0006_146.9,1.0,2307.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3168,AM,0.2011,6.1,8.8,2050,2300.0,2200,140.158656,Statewide,0,2500,COP,1.0,S00-Ann,140,0.984028,1-Weekday (Tu-Th),137,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,56.0741,1.0562,48.233133,59.225464,0.045289,0.055611
4,0065_000.1,1.0,614.1,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6182,AM,0.1791,2.9,1.5,2050,1800.0,1800,199.295316,Statewide,0,2500,COP,1.0,S00-Ann,199,0.984028,1-Weekday (Tu-Th),195,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,79.8135,1.0133,65.864500,80.875020,0.081718,0.100341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29089,0215_025.7,3.0,141936.9,Freeway,1.0,D2,5.0,34.0,Freeway,1784,0.4293,PM,0.2350,10.0,5.9,2050,111000.0,94000,9483.237000,I215,0,1000000,CO8,1.0,S00-Ann,9483,1.134873,1-Weekday (Tu-Th),10761,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,3938.5260,1.0495,3764.556500,4133.483037,0.422035,0.463395
29090,0215_026.7,3.0,59122.6,Freeway,1.0,D1,4.0,34.0,Freeway,1871,0.5447,PM,0.2521,9.9,6.9,2050,25500.0,23500,3226.993445,I215,0,1000000,CO8,1.0,S00-Ann,3226,1.134873,1-Weekday (Tu-Th),3661,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1339.9260,1.0543,1286.597433,1412.683982,0.171913,0.188761
29091,0215_026.7,3.0,59122.6,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4553,PM,0.2521,13.6,7.3,2050,25500.0,23500,2697.356555,I215,0,1000000,CO8,1.0,S00-Ann,2697,1.134873,1-Weekday (Tu-Th),3060,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1119.9600,1.0637,1084.974000,1191.301452,0.144972,0.159180
29092,0215_027.4,3.0,52448.7,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.5207,PM,0.2238,5.1,5.0,2050,34500.0,33500,3903.844110,I215,0,1000000,CO8,1.0,S00-Ann,3903,1.134873,1-Weekday (Tu-Th),4429,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1621.0140,1.0352,1528.300267,1678.073693,0.367380,0.403383


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,1917.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.5771,AM,0.2118,8.9,6.4,2032,1800.0,1800,220.013604,Statewide,0,2500,COP,1.0,S00-Ann,220,0.984028,1-Weekday (Tu-Th),216,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,88.4088,1.0498,75.585600,92.811558,0.070972,0.087147
1,0006_141.0,1.0,1917.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.4229,AM,0.2118,7.1,5.8,2032,1800.0,1800,161.226396,Statewide,0,2500,COP,1.0,S00-Ann,161,0.984028,1-Weekday (Tu-Th),158,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,64.6694,1.0432,54.941867,67.463118,0.051589,0.063346
2,0006_146.9,1.0,1926.8,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6339,AM,0.2112,9.3,6.6,2032,1800.0,1800,240.983424,Statewide,0,2500,COP,1.0,S00-Ann,240,0.984028,1-Weekday (Tu-Th),236,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,96.5948,1.0516,82.725867,101.579092,0.077677,0.095379
3,0006_146.9,1.0,1926.8,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3661,AM,0.2112,6.0,5.4,2032,1800.0,1800,139.176576,Statewide,0,2500,COP,1.0,S00-Ann,139,0.984028,1-Weekday (Tu-Th),136,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,55.6648,1.0390,47.101333,57.835727,0.044227,0.054306
4,0065_000.1,1.0,453.5,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6193,AM,0.1969,5.8,0.0,2032,1600.0,1600,195.104272,Statewide,0,2500,COP,1.0,S00-Ann,195,0.984028,1-Weekday (Tu-Th),191,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,78.1763,1.0116,64.405200,79.083145,0.079907,0.098118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28567,0215_025.7,3.0,144306.8,Freeway,1.0,D2,5.0,34.0,Freeway,1784,0.4420,PM,0.2532,12.6,7.2,2032,101000.0,101000,11303.354400,I215,0,1000000,CO8,1.0,S00-Ann,11303,1.134873,1-Weekday (Tu-Th),12827,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4694.6820,1.0612,4537.337467,4981.996538,0.508670,0.558520
28568,0215_026.7,3.0,61753.3,Freeway,1.0,D1,4.0,34.0,Freeway,1871,0.5372,PM,0.2523,19.2,9.3,2032,27500.0,27500,3727.227900,I215,0,1000000,CO8,1.0,S00-Ann,3727,1.134873,1-Weekday (Tu-Th),4229,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1547.8140,1.0849,1529.347367,1679.223409,0.204349,0.224375
28569,0215_026.7,3.0,61753.3,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4628,PM,0.2523,13.6,6.9,2032,27500.0,27500,3211.022100,I215,0,1000000,CO8,1.0,S00-Ann,3211,1.134873,1-Weekday (Tu-Th),3644,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1333.7040,1.0617,1289.611600,1415.993537,0.172316,0.189203
28570,0215_027.4,3.0,49902.1,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.5389,PM,0.2011,23.1,13.2,2032,33000.0,32500,3522.115675,I215,0,1000000,CO8,1.0,S00-Ann,3522,1.134873,1-Weekday (Tu-Th),3997,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1462.9020,1.1122,1481.821133,1627.039604,0.356207,0.391115


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,2151.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6106,AM,0.2060,8.1,7.3,2042,2100.0,2100,264.145560,Statewide,0,2500,COP,1.0,S00-Ann,264,0.984028,1-Weekday (Tu-Th),259,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,106.0087,1.0527,90.883100,111.595358,0.085336,0.104784
1,0006_141.0,1.0,2151.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3894,AM,0.2060,4.7,7.7,2042,2100.0,2100,168.454440,Statewide,0,2500,COP,1.0,S00-Ann,168,0.984028,1-Weekday (Tu-Th),165,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,67.5345,1.0479,57.634500,70.769403,0.054117,0.066450
2,0006_146.9,1.0,2157.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6914,AM,0.2058,8.8,7.2,2042,2100.0,2100,298.809252,Statewide,0,2500,COP,1.0,S00-Ann,298,0.984028,1-Weekday (Tu-Th),293,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,119.9249,1.0536,102.901600,126.352875,0.096621,0.118641
3,0006_146.9,1.0,2157.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3086,AM,0.2058,2.2,8.0,2042,2100.0,2100,133.370748,Statewide,0,2500,COP,1.0,S00-Ann,133,0.984028,1-Weekday (Tu-Th),130,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,53.2090,1.0444,45.257333,55.571480,0.042495,0.052180
4,0065_000.1,1.0,529.0,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6515,AM,0.1953,4.5,1.5,2042,1700.0,1700,216.304515,Statewide,0,2500,COP,1.0,S00-Ann,216,0.984028,1-Weekday (Tu-Th),212,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,86.7716,1.0165,71.832667,88.203331,0.089122,0.109433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29851,0215_025.7,3.0,160828.3,Freeway,1.0,D2,5.0,34.0,Freeway,1784,0.4333,PM,0.2562,11.8,6.2,2042,105000.0,106000,11767.214760,I215,0,1000000,CO8,1.0,S00-Ann,11767,1.134873,1-Weekday (Tu-Th),13354,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4887.5640,1.0546,4694.376133,5154.424994,0.526275,0.577850
29852,0215_026.7,3.0,64618.9,Freeway,1.0,D1,4.0,34.0,Freeway,1871,0.5552,PM,0.2597,17.5,9.2,2042,28500.0,27500,3965.099600,I215,0,1000000,CO8,1.0,S00-Ann,3965,1.134873,1-Weekday (Tu-Th),4499,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1646.6340,1.0810,1621.139667,1780.011354,0.216614,0.237842
29853,0215_026.7,3.0,64618.9,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4448,PM,0.2597,13.3,6.5,2042,28500.0,27500,3176.650400,I215,0,1000000,CO8,1.0,S00-Ann,3176,1.134873,1-Weekday (Tu-Th),3604,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1319.0640,1.0591,1272.332133,1397.020682,0.170007,0.186668
29854,0215_027.4,3.0,56849.7,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.5251,PM,0.2203,19.4,13.4,2042,35000.0,36500,4222.302845,I215,0,1000000,CO8,1.0,S00-Ann,4222,1.134873,1-Weekday (Tu-Th),4791,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1753.5060,1.1058,1765.962600,1939.026935,0.424510,0.466112


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,2346.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6183,AM,0.1974,9.0,7.5,2050,2300.0,2300,280.720566,Statewide,0,2500,COP,1.0,S00-Ann,280,0.984028,1-Weekday (Tu-Th),275,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,112.5575,1.0555,96.754167,118.804441,0.090849,0.111553
1,0006_141.0,1.0,2346.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3817,AM,0.1974,8.0,8.2,2050,2300.0,2300,173.299434,Statewide,0,2500,COP,1.0,S00-Ann,173,0.984028,1-Weekday (Tu-Th),170,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,69.5810,1.0570,59.896667,73.547117,0.056241,0.069058
2,0006_146.9,1.0,2351.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.7047,AM,0.1974,9.2,7.3,2050,2300.0,2300,319.947894,Statewide,0,2500,COP,1.0,S00-Ann,319,0.984028,1-Weekday (Tu-Th),313,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,128.1109,1.0549,110.061233,135.144188,0.103344,0.126896
3,0006_146.9,1.0,2351.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.2953,AM,0.1974,7.3,8.8,2050,2300.0,2300,134.072106,Statewide,0,2500,COP,1.0,S00-Ann,134,0.984028,1-Weekday (Tu-Th),131,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,53.6183,1.0586,46.225533,56.760332,0.043404,0.053296
4,0065_000.1,1.0,608.3,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6490,AM,0.1920,6.6,4.0,2050,1800.0,1800,224.294400,Statewide,0,2500,COP,1.0,S00-Ann,224,0.984028,1-Weekday (Tu-Th),220,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,90.0460,1.0332,75.768000,93.035527,0.094005,0.115429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31591,0215_025.7,3.0,168749.3,Freeway,1.0,D2,5.0,34.0,Freeway,1784,0.4207,PM,0.2483,10.4,6.3,2050,111000.0,112000,11699.498720,I215,0,1000000,CO8,1.0,S00-Ann,11699,1.134873,1-Weekday (Tu-Th),13276,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4859.0160,1.0523,4656.778267,5113.142537,0.522060,0.573222
31592,0215_026.7,3.0,66134.1,Freeway,1.0,D1,4.0,34.0,Freeway,1871,0.5845,PM,0.2563,16.6,9.8,2050,25500.0,26500,3969.894775,I215,0,1000000,CO8,1.0,S00-Ann,3969,1.134873,1-Weekday (Tu-Th),4504,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1648.4640,1.0822,1624.742933,1783.967741,0.217096,0.238371
31593,0215_026.7,3.0,66134.1,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4155,PM,0.2563,13.2,7.4,2050,25500.0,26500,2822.055225,I215,0,1000000,CO8,1.0,S00-Ann,2822,1.134873,1-Weekday (Tu-Th),3202,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1171.9320,1.0634,1135.002267,1246.232489,0.151657,0.166520
31594,0215_027.4,3.0,58652.9,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.5818,PM,0.2251,16.5,10.3,2050,34500.0,37500,4911.119250,I215,0,1000000,CO8,1.0,S00-Ann,4911,1.134873,1-Weekday (Tu-Th),5573,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,2039.7180,1.0845,2014.639500,2212.074171,0.484288,0.531749


,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6145,AM,0.1973,9.0,7.6,2050,2300.0,2300,278.853955,Statewide,0,2500,COP,1.0,S00-Ann,278,0.984028,1-Weekday (Tu-Th),273,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,111.7389,1.0560,96.096000,117.996278,0.090231,0.110795
1,0006_141.0,1.0,2362.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3855,AM,0.1973,7.9,8.5,2050,2300.0,2300,174.936045,Statewide,0,2500,COP,1.0,S00-Ann,174,0.984028,1-Weekday (Tu-Th),171,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,69.9903,1.0583,60.323100,74.070734,0.056641,0.069550
2,0006_146.9,1.0,2365.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6981,AM,0.1975,9.2,7.4,2050,2300.0,2300,317.111925,Statewide,0,2500,COP,1.0,S00-Ann,317,0.984028,1-Weekday (Tu-Th),311,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,127.2923,1.0554,109.409800,134.344293,0.102732,0.126145
3,0006_146.9,1.0,2365.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3019,AM,0.1975,7.1,9.2,2050,2300.0,2300,137.138075,Statewide,0,2500,COP,1.0,S00-Ann,137,0.984028,1-Weekday (Tu-Th),134,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,54.8462,1.0602,47.355600,58.147941,0.044465,0.054599
4,0065_000.1,1.0,606.3,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6303,AM,0.1829,4.3,1.4,2050,1800.0,1800,207.507366,Statewide,0,2500,COP,1.0,S00-Ann,207,0.984028,1-Weekday (Tu-Th),203,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,83.0879,1.0156,68.722267,84.384071,0.085263,0.104695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31597,0215_025.7,3.0,168159.4,Freeway,1.0,D2,5.0,34.0,Freeway,1784,0.4204,PM,0.2479,10.4,6.3,2050,111000.0,112000,11672.321920,I215,0,1000000,CO8,1.0,S00-Ann,11672,1.134873,1-Weekday (Tu-Th),13246,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4848.0360,1.0523,4646.255267,5101.588283,0.520881,0.571927
31598,0215_026.7,3.0,65775.5,Freeway,1.0,D1,4.0,34.0,Freeway,1871,0.5857,PM,0.2546,16.9,9.9,2050,25500.0,26500,3951.659330,I215,0,1000000,CO8,1.0,S00-Ann,3951,1.134873,1-Weekday (Tu-Th),4483,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1640.7780,1.0833,1618.811300,1777.454807,0.216303,0.237501
31599,0215_026.7,3.0,65775.5,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4143,PM,0.2546,13.3,7.8,2050,25500.0,26500,2795.240670,I215,0,1000000,CO8,1.0,S00-Ann,2795,1.134873,1-Weekday (Tu-Th),3171,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1160.5860,1.0656,1126.339200,1236.720442,0.150500,0.165249
31600,0215_027.4,3.0,58587.8,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.5822,PM,0.2219,16.5,10.6,2050,34500.0,37500,4844.631750,I215,0,1000000,CO8,1.0,S00-Ann,4844,1.134873,1-Weekday (Tu-Th),5497,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,2011.9020,1.0860,1989.914000,2184.925572,0.478345,0.525222


In [328]:
df_PkHrForecasts.columns

Index(['SEGID', 'AREATYPE', 'SegDYVol_TDM', 'FUNCGROUP', 'FGFAC', 'Dir',
       'LANES', 'FT', 'FTCLASS', 'CAP1HL', 'DFAC', 'Prd_TDM', 'PRDFAC',
       'MDPCT', 'HVPCT', 'FYEAR', 'ForecastAADT', 'ForecastAADTAdj',
       'AvgAnnualPrdVol', 'FACGEO', 'VOLUMEFROM', 'VOLUMETO', 'STATIONGROUP',
       'SsnFactor', 'SEASONGROUP', 'AvgSsnPrdVol', 'DOWFactor', 'DOWGROUP',
       'AvgSsnDOWPrdVol', 'YEARGROUP', 'PERIOD', 'VOLPKHRPCT', 'HRPCTOF',
       'SeasonType', 'VCGroupCode', 'PRDHRS', 'AvgSsnDOWPkHrVol', 'TrkFac',
       'PrdPCEFlow', 'PkHrPCEFlow', 'PrdVC', 'PkHrVC'],
      dtype='object')

In [329]:
df_PkHrForecasts[df_PkHrForecasts['SEASONGROUP']=='S00-Ann']

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC
0,0006_141.0,1.0,2362.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6145,AM,0.1973,9.0,7.6,2050,2300.0,2300,278.853955,Statewide,0,2500,COP,1.0,S00-Ann,278,0.984028,1-Weekday (Tu-Th),273,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,111.7389,1.0560,96.096000,117.996278,0.090231,0.110795
1,0006_141.0,1.0,2362.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3855,AM,0.1973,7.9,8.5,2050,2300.0,2300,174.936045,Statewide,0,2500,COP,1.0,S00-Ann,174,0.984028,1-Weekday (Tu-Th),171,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,69.9903,1.0583,60.323100,74.070734,0.056641,0.069550
2,0006_146.9,1.0,2365.0,Arterial,1.0,D1,1.0,2.0,Principal Arterial,1065,0.6981,AM,0.1975,9.2,7.4,2050,2300.0,2300,317.111925,Statewide,0,2500,COP,1.0,S00-Ann,317,0.984028,1-Weekday (Tu-Th),311,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,127.2923,1.0554,109.409800,134.344293,0.102732,0.126145
3,0006_146.9,1.0,2365.0,Arterial,1.0,D2,1.0,2.0,Principal Arterial,1065,0.3019,AM,0.1975,7.1,9.2,2050,2300.0,2300,137.138075,Statewide,0,2500,COP,1.0,S00-Ann,137,0.984028,1-Weekday (Tu-Th),134,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,54.8462,1.0602,47.355600,58.147941,0.044465,0.054599
4,0065_000.1,1.0,606.3,Arterial,1.0,D1,1.0,4.0,Collector,806,0.6303,AM,0.1829,4.3,1.4,2050,1800.0,1800,207.507366,Statewide,0,2500,COP,1.0,S00-Ann,207,0.984028,1-Weekday (Tu-Th),203,2015-2019,AM,0.4093,Prd,Season,AnnWkAM,3,83.0879,1.0156,68.722267,84.384071,0.085263,0.104695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31597,0215_025.7,3.0,168159.4,Freeway,1.0,D2,5.0,34.0,Freeway,1784,0.4204,PM,0.2479,10.4,6.3,2050,111000.0,112000,11672.321920,I215,0,1000000,CO8,1.0,S00-Ann,11672,1.134873,1-Weekday (Tu-Th),13246,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,4848.0360,1.0523,4646.255267,5101.588283,0.520881,0.571927
31598,0215_026.7,3.0,65775.5,Freeway,1.0,D1,4.0,34.0,Freeway,1871,0.5857,PM,0.2546,16.9,9.9,2050,25500.0,26500,3951.659330,I215,0,1000000,CO8,1.0,S00-Ann,3951,1.134873,1-Weekday (Tu-Th),4483,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1640.7780,1.0833,1618.811300,1777.454807,0.216303,0.237501
31599,0215_026.7,3.0,65775.5,Freeway,1.0,D2,4.0,34.0,Freeway,1871,0.4143,PM,0.2546,13.3,7.8,2050,25500.0,26500,2795.240670,I215,0,1000000,CO8,1.0,S00-Ann,2795,1.134873,1-Weekday (Tu-Th),3171,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,1160.5860,1.0656,1126.339200,1236.720442,0.150500,0.165249
31600,0215_027.4,3.0,58587.8,Freeway,1.0,D1,2.0,34.0,Freeway,2080,0.5822,PM,0.2219,16.5,10.6,2050,34500.0,37500,4844.631750,I215,0,1000000,CO8,1.0,S00-Ann,4844,1.134873,1-Weekday (Tu-Th),5497,2015-2019,PM,0.3660,Prd,Season,AnnWkPM,3,2011.9020,1.0860,1989.914000,2184.925572,0.478345,0.525222


In [330]:
#15-Min Analysis

dfs_VC = []

for df_PkHrForecasts in dfs_PkHrForecasts:

    df_VC = df_PkHrForecasts
    
    #PHF conditions (from Chad's spreadsheet for Urban (0.92,0.95,0.97,0.98). HCM2000 Exhibit 9.2 says default PHF for Urban Areas is 0.92, Rural is 0.88. Rural estimated at (0.88,0.91,0.93,0.95)
    conditions = [
        (df_VC[nameAT] >= 3) & (df_VC['PkHrVC'] <= 0.85),
        (df_VC[nameAT] >= 3) & (df_VC['PkHrVC'] >  0.85) & (df_VC['PkHrVC'] <= 0.95),
        (df_VC[nameAT] >= 3) & (df_VC['PkHrVC'] >  0.95) & (df_VC['PkHrVC'] <= 1.00),
        (df_VC[nameAT] >= 3) & (df_VC['PkHrVC'] >  1.00),
        (df_VC[nameAT] <= 2) & (df_VC['PkHrVC'] <= 0.85),
        (df_VC[nameAT] <= 2) & (df_VC['PkHrVC'] >  0.85) & (df_VC['PkHrVC'] <= 0.95),
        (df_VC[nameAT] <= 2) & (df_VC['PkHrVC'] >  0.95) & (df_VC['PkHrVC'] <= 1.00),
        (df_VC[nameAT] <= 2) & (df_VC['PkHrVC'] >  1.00)
    ]
    choices = [0.92,0.95,0.97,0.98,0.88,0.91,0.93,0.95]

    df_VC['PHF'] = np.select(conditions, choices, default=0)


    #15-min volume and v/c calcs
    df_VC['Pk15PCEFlow'] = df_VC['PkHrPCEFlow']  / df_VC['PHF']
    df_VC['Pk15VC'     ] = df_VC['Pk15PCEFlow'] / (df_VC[nameCap1HL] * df_VC[nameLanes])

    #effective K-factor
    #df_VC['kFactor_effective']  = df_VC['ForecastPkHr']  / df_VC['ForecastAADT']

    #convert to int and round columns after calcs
    df_VC = df_VC.fillna(0)
    df_VC['AREATYPE'       ] = df_VC['AREATYPE'       ].astype(int)
    df_VC['FT'             ] = df_VC['FT'             ].astype(int)
    df_VC['LANES'          ] = df_VC['LANES'          ].astype(int)
    df_VC['ForecastAADT'   ] = df_VC['ForecastAADT'   ].astype(int)
    df_VC['ForecastAADTAdj'] = df_VC['ForecastAADTAdj'].astype(int)
    df_VC['PrdPCEFlow'     ] = df_VC['PrdPCEFlow'     ].astype(int)
    df_VC['PkHrPCEFlow'    ] = df_VC['PkHrPCEFlow'    ].astype(int)
    df_VC['Pk15PCEFlow'    ] = df_VC['Pk15PCEFlow'    ].astype(int)
    df_VC = df_VC.round({'SeasonFactor':2,'DOWFactor':2,'PrdVC':2,'PkHrVC':2,'Pk15VC':2,nameMDPct:2,nameHVPct:2,'VolMaxHrPct':2,'FactorTrucks':2})
    
    pd.set_option('display.max_columns', None)
    
    #display(df_VC)
    
    dfs_VC.append(df_VC)
    

In [331]:
df = dfs_VC[0]

df = df[(df[nameSegID]=='0015_282.2') * (df['VCGroupCode']=='AnnWkPM')]

df

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FAC_WDAVG,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
19402,0015_282.2,4,210915.6,Arterial,0.0405,D1,2,3,Minor Arterial,978,0.5654,PM,0.2867,15.7,7.5,1.0654,2023,183000,183000,1201.405484,I15,0,1000000,COU,1.0,S00-Ann,1201,1.09,1-Weekday (Tu-Th),1314,2015-2019,PM,0.3615,Prd,Season,AnnWkPM,3,475.011,1.0689,468,507,0.24,0.26,0.92,551,0.28
19403,0015_282.2,4,210915.6,Arterial,0.0405,D2,1,3,Minor Arterial,1019,0.4346,PM,0.2867,8.1,6.3,1.0654,2023,183000,183000,923.471566,I15,0,1000000,COU,1.0,S00-Ann,923,1.09,1-Weekday (Tu-Th),1010,2015-2019,PM,0.3615,Prd,Season,AnnWkPM,3,365.115,1.0477,352,382,0.35,0.38,0.92,415,0.41
26092,0015_282.2,4,210915.6,Freeway,0.7511,D1,5,33,Freeway,1644,0.4765,PM,0.2513,8.7,6.7,1.0654,2023,183000,183000,16459.030320,I15,0,1000000,CO6,1.0,S00-Ann,16459,1.07,1-Weekday (Tu-Th),17534,2015-2019,PM,0.3510,Prd,Season,AnnWkPM,3,6154.434,1.0509,6142,6467,0.75,0.79,0.92,7030,0.86
26093,0015_282.2,4,210915.6,Freeway,0.7511,D2,4,33,Freeway,1696,0.5235,PM,0.2513,7.6,5.8,1.0654,2023,183000,183000,18082.481370,I15,0,1000000,CO6,1.0,S00-Ann,18082,1.07,1-Weekday (Tu-Th),19263,2015-2019,PM,0.3510,Prd,Season,AnnWkPM,3,6761.313,1.0442,6704,7060,0.99,1.04,0.98,7204,1.06
26094,0015_282.2,4,210915.6,Managed,0.1027,D1,1,38,Freeway,1767,0.4401,PM,0.3042,0.0,0.0,1.0654,2023,183000,183000,2516.124413,I15,0,1000000,CO6,1.0,S00-Ann,2516,1.07,1-Weekday (Tu-Th),2680,2015-2019,PM,0.3510,Prd,Season,AnnWkPM,3,940.680,1.0000,893,940,0.51,0.53,0.92,1022,0.58
26095,0015_282.2,4,210915.6,Managed,0.1027,D2,1,38,Freeway,1767,0.5599,PM,0.3042,0.0,0.0,1.0654,2023,183000,183000,3201.040807,I15,0,1000000,CO6,1.0,S00-Ann,3201,1.07,1-Weekday (Tu-Th),3410,2015-2019,PM,0.3510,Prd,Season,AnnWkPM,3,1196.910,1.0000,1136,1196,0.64,0.68,0.92,1300,0.74
26096,0015_282.2,4,210915.6,CD Road,0.1057,D1,2,31,Freeway,1473,0.4693,PM,0.2862,14.0,9.9,1.0654,2023,183000,183000,2598.042557,I15,0,1000000,CO6,1.0,S00-Ann,2598,1.07,1-Weekday (Tu-Th),2767,2015-2019,PM,0.3510,Prd,Season,AnnWkPM,3,971.217,1.0775,993,1046,0.34,0.36,0.92,1137,0.39
26097,0015_282.2,4,210915.6,CD Road,0.1057,D2,1,31,Freeway,1473,0.5307,PM,0.2862,9.8,6.2,1.0654,2023,183000,183000,2937.952663,I15,0,1000000,CO6,1.0,S00-Ann,2937,1.07,1-Weekday (Tu-Th),3128,2015-2019,PM,0.3510,Prd,Season,AnnWkPM,3,1097.928,1.0506,1095,1153,0.74,0.78,0.92,1253,0.85


In [332]:
df_VC.columns

Index(['SEGID', 'AREATYPE', 'SegDYVol_TDM', 'FUNCGROUP', 'FGFAC', 'Dir',
       'LANES', 'FT', 'FTCLASS', 'CAP1HL', 'DFAC', 'Prd_TDM', 'PRDFAC',
       'MDPCT', 'HVPCT', 'FYEAR', 'ForecastAADT', 'ForecastAADTAdj',
       'AvgAnnualPrdVol', 'FACGEO', 'VOLUMEFROM', 'VOLUMETO', 'STATIONGROUP',
       'SsnFactor', 'SEASONGROUP', 'AvgSsnPrdVol', 'DOWFactor', 'DOWGROUP',
       'AvgSsnDOWPrdVol', 'YEARGROUP', 'PERIOD', 'VOLPKHRPCT', 'HRPCTOF',
       'SeasonType', 'VCGroupCode', 'PRDHRS', 'AvgSsnDOWPkHrVol', 'TrkFac',
       'PrdPCEFlow', 'PkHrPCEFlow', 'PrdVC', 'PkHrVC', 'PHF', 'Pk15PCEFlow',
       'Pk15VC'],
      dtype='object')

In [333]:
df1 = df_VC.groupby(['SEGID'                  ],as_index=False).agg(num=(nameDir,'size'))
df2 = df_VC.groupby(['SEGID','ForecastAADTAdj'],as_index=False).agg(num=(nameDir,'size'))
df3 = pd.DataFrame.merge(df1,df2,on='SEGID',how='outer')

In [334]:
df2g = df2.groupby(['SEGID'],as_index=False).agg(numFor=('ForecastAADTAdj','size'))
df2g[df2g['numFor']==2]


,SEGID,numFor


In [335]:
df2[df2['SEGID']=='0038_003.2']

,SEGID,ForecastAADTAdj,num
172,0038_003.2,5500,6


In [336]:
dfs_VC_MaxDirFG = []

for df_VC in dfs_VC:

    df_VC_MaxDirFG = df_VC.groupby([nameSegID,'PERIOD','SEASONGROUP','DOWGROUP','HRPCTOF','VCGroupCode'], as_index=False).agg({nameDir:[np.size],nameCap1HL:[np.sum],'PrdPCEFlow':[np.sum],'PkHrPCEFlow':[np.sum],'Pk15PCEFlow':[np.sum],'PrdVC':[np.max],'PkHrVC':[np.max],'Pk15VC':[np.max]})

    df_VC_MaxDirFG.columns = ([nameSegID,'PERIOD','SEASONGROUP','DOWGROUP','HRPCTOF','VCGroupCode','numRecords',nameCap1HL+'Sum','PrdPCEFlowSum','PkHrPCEFlowSum','Pk15PCEFlowSum','PrdVCMax','PkHrVCMax','Pk15VCMax'])
    
    dfs_VC_MaxDirFG.append(df_VC_MaxDirFG)
    
    #display(df_VC_MaxDirFG)
                                            
    #df_Peak_VC_wGroups = pd.DataFrame.merge(df_VCGroups,df_Peak_VC,on=('SeasonGroup','DOWGroup','Prd','HrPctOf'))

In [337]:
display(df_Scenarios)

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Base19,2019 Base,2019,Base,v901_SE19_Net19_Summary_SEGID_Detailed.csv,No,NaN,NaN
1,Base23,2023 Base,2023,Base,v901_SE23_Net23_Summary_SEGID_Detailed.csv,Yes,Base19,NaN
2,TIP28,2028 TIP Projects,2028,TIP,v901_TIP_SE28_Net28_Summary_SEGID_Detailed.csv,Yes,Base23,NaN
3,RTP32,2032 Fiscally-Constrained,2032,RTP,v901_RTP_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,NaN
4,RTP42,2042 Fiscally-Constrained,2042,RTP,v901_RTP_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,RTP32,NaN
...,...,...,...,...,...,...,...,...
11,42NB50,2050 SE on a 2042 Network,2050,NoBuild42,v901_NoBuild_SE50_Net42_Summary_SEGID_Detailed...,Yes,RTP42,RTP50
12,UF32,2032 Needs-Based,2032,Needs,v901_Needs_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,RTP32
13,UF42,2042 Needs-Based,2042,Needs,v901_Needs_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,UF32,RTP42


In [338]:
df = dfs_VC[0]

df = df[(df[nameSegID]=='0015_295.6') * (df['VCGroupCode']=='AnnWkPM')]

display(df)

df = dfs_VC_MaxDirFG[0]

df = df[(df[nameSegID]=='0015_295.6')]

pd.set_option('display.max_columns', None)

display(df)

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPCT,HVPCT,FAC_WDAVG,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,FACGEO,VOLUMEFROM,VOLUMETO,STATIONGROUP,SsnFactor,SEASONGROUP,AvgSsnPrdVol,DOWFactor,DOWGROUP,AvgSsnDOWPrdVol,YEARGROUP,PERIOD,VOLPKHRPCT,HRPCTOF,SeasonType,VCGroupCode,PRDHRS,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
26144,0015_295.6,4,354322.4,Freeway,0.6418,D1,5,34,Freeway,1784,0.3845,PM,0.2063,12.3,7.1,1.0654,2023,315000,315000,16036.361532,I15,0,1000000,CO6,1.0,S00-Ann,16036,1.07,1-Weekday (Tu-Th),17084,2015-2019,PM,0.351,Prd,Season,AnnWkPM,3,5996.484,1.0601,6036,6356,0.68,0.71,0.92,6909,0.77
26145,0015_295.6,4,354322.4,Freeway,0.6418,D2,5,34,Freeway,1784,0.6155,PM,0.2063,8.4,5.5,1.0654,2023,315000,315000,25670.690568,I15,0,1000000,CO6,1.0,S00-Ann,25670,1.07,1-Weekday (Tu-Th),27347,2015-2019,PM,0.351,Prd,Season,AnnWkPM,3,9598.797,1.0443,9519,10024,1.07,1.12,0.98,10228,1.15
26146,0015_295.6,4,354322.4,Managed,0.0948,D1,1,38,Freeway,1767,0.4220,PM,0.3029,0.0,0.0,1.0654,2023,315000,315000,3817.074316,I15,0,1000000,CO6,1.0,S00-Ann,3817,1.07,1-Weekday (Tu-Th),4066,2015-2019,PM,0.351,Prd,Season,AnnWkPM,3,1427.166,1.0000,1355,1427,0.77,0.81,0.92,1551,0.88
26147,0015_295.6,4,354322.4,Managed,0.0948,D2,1,38,Freeway,1767,0.5780,PM,0.3029,0.0,0.0,1.0654,2023,315000,315000,5228.125484,I15,0,1000000,CO6,1.0,S00-Ann,5228,1.07,1-Weekday (Tu-Th),5569,2015-2019,PM,0.351,Prd,Season,AnnWkPM,3,1954.719,1.0000,1856,1954,1.05,1.11,0.98,1994,1.13
26148,0015_295.6,4,354322.4,CD Road,0.2634,D1,2,31,Freeway,1473,0.4321,PM,0.2045,9.8,7.0,1.0654,2023,315000,315000,7331.686781,I15,0,1000000,CO6,1.0,S00-Ann,7331,1.07,1-Weekday (Tu-Th),7810,2015-2019,PM,0.351,Prd,Season,AnnWkPM,3,2741.310,1.0546,2745,2890,0.93,0.98,0.97,2980,1.01
26149,0015_295.6,4,354322.4,CD Road,0.2634,D2,3,31,Freeway,1473,0.5679,PM,0.2045,7.8,5.7,1.0654,2023,315000,315000,9635.882719,I15,0,1000000,CO6,1.0,S00-Ann,9635,1.07,1-Weekday (Tu-Th),10264,2015-2019,PM,0.351,Prd,Season,AnnWkPM,3,3602.664,1.0441,3572,3761,0.81,0.85,0.95,3959,0.90


,SEGID,PERIOD,SEASONGROUP,DOWGROUP,HRPCTOF,VCGroupCode,numRecords,CAP1HLSum,PrdPCEFlowSum,PkHrPCEFlowSum,Pk15PCEFlowSum,PrdVCMax,PkHrVCMax,Pk15VCMax
267,0015_295.6,AM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkAM,6,10048,21413,23776,25318,0.92,1.02,1.04
268,0015_295.6,MD,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkMD,6,10048,19469,21856,23376,0.84,0.94,0.99
269,0015_295.6,PM,S00-Ann,1-Weekday (Tu-Th),Prd,AnnWkPM,6,10048,25083,26412,27621,1.07,1.12,1.15


# VC Groups as Columns

In [339]:
df_VCGroups

,SEASONGROUP,SeasonType,DOWGROUP,PERIOD,HRPCTOF,VCGroupCode
VCGroupID,,,,,,
0,S00-Ann,Season,1-Weekday (Tu-Th),AM,Prd,AnnWkAM
1,S00-Ann,Season,1-Weekday (Tu-Th),MD,Prd,AnnWkMD
2,S00-Ann,Season,1-Weekday (Tu-Th),PM,Prd,AnnWkPM


In [340]:
dfs_VC_MaxDirFG_wVCGroups_15 = []

for df_VC_MaxDirFG in dfs_VC_MaxDirFG:

    df_VC_MaxDirFG_wVCGroups_15 = df_VC_MaxDirFG[['VCGroupCode','SEGID','Pk15VCMax']].copy()
    
    df_VC_MaxDirFG_wVCGroups_15['VCGroupCode'] = df_VC_MaxDirFG_wVCGroups_15['VCGroupCode'] + "15"
    
    df_VC_MaxDirFG_wVCGroups_15 = df_VC_MaxDirFG_wVCGroups_15.round({'Pk15VC': 2})
    
    df_VC_MaxDirFG_wVCGroups_15 = df_VC_MaxDirFG_wVCGroups_15.pivot(index='SEGID', columns="VCGroupCode",values='Pk15VCMax')
    
    df_VC_MaxDirFG_wVCGroups_15 = df_VC_MaxDirFG_wVCGroups_15.reset_index()
    
    dfs_VC_MaxDirFG_wVCGroups_15.append(df_VC_MaxDirFG_wVCGroups_15)

    display(df_VC_MaxDirFG_wVCGroups_15)
    

VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_141.0,0.09,0.06,0.08
1,0006_146.9,0.10,0.06,0.08
2,0006_149.9,0.13,0.09,0.18
3,0006_150.6,0.12,0.09,0.17
4,0006_152.6,0.14,0.09,0.18
...,...,...,...,...
4314,WFRC_8461,0.20,0.16,0.24
4315,WFRC_8466,0.37,0.21,0.40
4316,WFRC_8467,0.55,0.32,0.67
4317,WFRC_8473,0.61,0.54,0.78


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_141.0,0.09,0.06,0.08
1,0006_146.9,0.11,0.07,0.09
2,0006_149.9,0.14,0.10,0.21
3,0006_150.6,0.14,0.10,0.20
4,0006_152.6,0.16,0.10,0.21
...,...,...,...,...
4381,WFRC_8461,0.20,0.14,0.43
4382,WFRC_8466,0.42,0.23,0.45
4383,WFRC_8467,0.68,0.38,0.79
4384,WFRC_8473,0.62,0.55,0.75


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_141.0,0.10,0.07,0.09
1,0006_146.9,0.11,0.07,0.09
2,0006_149.9,0.15,0.10,0.20
3,0006_150.6,0.15,0.10,0.20
4,0006_152.6,0.17,0.11,0.19
...,...,...,...,...
4573,WFRC_8466,0.44,0.25,0.48
4574,WFRC_8467,0.68,0.40,0.86
4575,WFRC_8471,0.51,0.38,0.63
4576,WFRC_8473,0.67,0.57,0.82


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_141.0,0.12,0.08,0.12
1,0006_146.9,0.13,0.08,0.12
2,0006_149.9,0.21,0.11,0.22
3,0006_150.6,0.21,0.12,0.23
4,0006_152.6,0.23,0.13,0.24
...,...,...,...,...
4668,WFRC_8466,0.51,0.32,0.53
4669,WFRC_8467,0.70,0.48,0.76
4670,WFRC_8471,0.63,0.46,0.76
4671,WFRC_8473,0.74,0.64,0.85


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_141.0,0.10,0.07,0.09
1,0006_146.9,0.11,0.07,0.09
2,0006_149.9,0.18,0.11,0.24
3,0006_150.6,0.18,0.11,0.24
4,0006_152.6,0.20,0.12,0.25
...,...,...,...,...
4382,WFRC_8461,0.28,0.18,0.58
4383,WFRC_8466,0.45,0.25,0.48
4384,WFRC_8467,0.77,0.42,0.84
4385,WFRC_8473,0.68,0.59,0.79


...

VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_141.0,0.12,0.08,0.11
1,0006_146.9,0.14,0.09,0.11
2,0006_149.9,0.21,0.12,0.25
3,0006_150.6,0.20,0.13,0.25
4,0006_152.6,0.25,0.16,0.30
...,...,...,...,...
4675,WFRC_8466,0.56,0.37,0.58
4676,WFRC_8467,0.69,0.50,0.75
4677,WFRC_8471,0.64,0.50,0.79
4678,WFRC_8473,0.82,0.68,0.91


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_141.0,0.10,0.07,0.10
1,0006_146.9,0.11,0.07,0.10
2,0006_149.9,0.14,0.09,0.19
3,0006_150.6,0.14,0.10,0.19
4,0006_152.6,0.16,0.11,0.18
...,...,...,...,...
4605,WFRC_8466,0.44,0.24,0.48
4606,WFRC_8467,0.68,0.39,0.77
4607,WFRC_8471,0.50,0.41,0.64
4608,WFRC_8473,0.67,0.57,0.81


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_141.0,0.12,0.08,0.11
1,0006_146.9,0.13,0.08,0.12
2,0006_149.9,0.15,0.11,0.41
3,0006_150.6,0.15,0.12,0.40
4,0006_152.6,0.16,0.12,0.35
...,...,...,...,...
4744,WFRC_8470,0.41,0.28,0.65
4745,WFRC_8471,0.65,0.52,0.81
4746,WFRC_8472,0.50,0.61,1.02
4747,WFRC_8473,0.74,0.61,0.83


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_141.0,0.13,0.08,0.13
1,0006_146.9,0.14,0.09,0.13
2,0006_149.9,0.17,0.15,0.26
3,0006_150.6,0.17,0.15,0.26
4,0006_152.6,0.21,0.17,0.27
...,...,...,...,...
5031,WFRC_8470,0.41,0.28,0.43
5032,WFRC_8471,0.64,0.55,0.80
5033,WFRC_8472,0.49,0.70,1.03
5034,WFRC_8473,0.87,0.68,0.95


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15
0,0006_141.0,0.13,0.08,0.13
1,0006_146.9,0.14,0.09,0.13
2,0006_149.9,0.16,0.14,0.26
3,0006_150.6,0.17,0.15,0.26
4,0006_152.6,0.21,0.17,0.27
...,...,...,...,...
5032,WFRC_8470,0.41,0.29,0.44
5033,WFRC_8471,0.64,0.55,0.80
5034,WFRC_8472,0.50,0.71,1.02
5035,WFRC_8473,0.88,0.68,0.94


In [341]:
df_VC_MaxDirFG.columns

Index(['SEGID', 'PERIOD', 'SEASONGROUP', 'DOWGROUP', 'HRPCTOF', 'VCGroupCode',
       'numRecords', 'CAP1HLSum', 'PrdPCEFlowSum', 'PkHrPCEFlowSum',
       'Pk15PCEFlowSum', 'PrdVCMax', 'PkHrVCMax', 'Pk15VCMax'],
      dtype='object')

In [342]:
dfs_VC_MaxDirFG_wVCGroups_Hr = []

for df_VC_MaxDirFG in dfs_VC_MaxDirFG:

    df_VC_MaxDirFG_wVCGroups_Hr = df_VC_MaxDirFG[['VCGroupCode','SEGID','PkHrVCMax']].copy()
    
    df_VC_MaxDirFG_wVCGroups_Hr['VCGroupCode'] = df_VC_MaxDirFG_wVCGroups_Hr['VCGroupCode'] + "Hr"
    
    df_VC_MaxDirFG_wVCGroups_Hr = df_VC_MaxDirFG_wVCGroups_Hr.round({'PkHrVC': 2})
    
    df_VC_MaxDirFG_wVCGroups_Hr = df_VC_MaxDirFG_wVCGroups_Hr.pivot(index=('SEGID'), columns="VCGroupCode",values='PkHrVCMax')
    
    df_VC_MaxDirFG_wVCGroups_Hr = df_VC_MaxDirFG_wVCGroups_Hr.reset_index()
    
    dfs_VC_MaxDirFG_wVCGroups_Hr.append(df_VC_MaxDirFG_wVCGroups_Hr)

    display(df_VC_MaxDirFG_wVCGroups_Hr)
    

VCGroupCode,SEGID,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr
0,0006_141.0,0.08,0.05,0.07
1,0006_146.9,0.09,0.05,0.07
2,0006_149.9,0.11,0.08,0.16
3,0006_150.6,0.11,0.08,0.15
4,0006_152.6,0.13,0.08,0.16
...,...,...,...,...
4314,WFRC_8461,0.18,0.14,0.21
4315,WFRC_8466,0.34,0.19,0.37
4316,WFRC_8467,0.51,0.29,0.62
4317,WFRC_8473,0.56,0.50,0.71


VCGroupCode,SEGID,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr
0,0006_141.0,0.08,0.05,0.07
1,0006_146.9,0.10,0.06,0.08
2,0006_149.9,0.12,0.09,0.18
3,0006_150.6,0.12,0.09,0.18
4,0006_152.6,0.14,0.09,0.18
...,...,...,...,...
4381,WFRC_8461,0.18,0.13,0.40
4382,WFRC_8466,0.39,0.21,0.41
4383,WFRC_8467,0.63,0.35,0.73
4384,WFRC_8473,0.57,0.50,0.69


VCGroupCode,SEGID,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr
0,0006_141.0,0.09,0.06,0.08
1,0006_146.9,0.10,0.06,0.08
2,0006_149.9,0.13,0.08,0.18
3,0006_150.6,0.13,0.09,0.18
4,0006_152.6,0.15,0.10,0.17
...,...,...,...,...
4573,WFRC_8466,0.40,0.23,0.44
4574,WFRC_8467,0.63,0.36,0.79
4575,WFRC_8471,0.47,0.35,0.58
4576,WFRC_8473,0.62,0.52,0.75


VCGroupCode,SEGID,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr
0,0006_141.0,0.10,0.07,0.10
1,0006_146.9,0.12,0.07,0.10
2,0006_149.9,0.19,0.10,0.20
3,0006_150.6,0.19,0.10,0.20
4,0006_152.6,0.20,0.11,0.21
...,...,...,...,...
4668,WFRC_8466,0.47,0.29,0.48
4669,WFRC_8467,0.65,0.44,0.70
4670,WFRC_8471,0.58,0.43,0.70
4671,WFRC_8473,0.68,0.59,0.78


VCGroupCode,SEGID,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr
0,0006_141.0,0.09,0.06,0.08
1,0006_146.9,0.10,0.06,0.08
2,0006_149.9,0.16,0.09,0.21
3,0006_150.6,0.16,0.09,0.21
4,0006_152.6,0.17,0.10,0.22
...,...,...,...,...
4382,WFRC_8461,0.26,0.17,0.53
4383,WFRC_8466,0.41,0.23,0.44
4384,WFRC_8467,0.71,0.39,0.77
4385,WFRC_8473,0.63,0.54,0.73


...

VCGroupCode,SEGID,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr
0,0006_141.0,0.11,0.07,0.10
1,0006_146.9,0.12,0.08,0.10
2,0006_149.9,0.18,0.11,0.22
3,0006_150.6,0.18,0.11,0.22
4,0006_152.6,0.22,0.14,0.26
...,...,...,...,...
4675,WFRC_8466,0.52,0.34,0.53
4676,WFRC_8467,0.64,0.46,0.69
4677,WFRC_8471,0.59,0.46,0.72
4678,WFRC_8473,0.76,0.63,0.86


VCGroupCode,SEGID,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr
0,0006_141.0,0.09,0.06,0.09
1,0006_146.9,0.10,0.06,0.09
2,0006_149.9,0.13,0.08,0.16
3,0006_150.6,0.13,0.08,0.17
4,0006_152.6,0.14,0.09,0.16
...,...,...,...,...
4605,WFRC_8466,0.41,0.22,0.44
4606,WFRC_8467,0.62,0.36,0.71
4607,WFRC_8471,0.46,0.37,0.59
4608,WFRC_8473,0.61,0.53,0.74


VCGroupCode,SEGID,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr
0,0006_141.0,0.10,0.07,0.10
1,0006_146.9,0.12,0.07,0.10
2,0006_149.9,0.13,0.10,0.36
3,0006_150.6,0.13,0.10,0.36
4,0006_152.6,0.14,0.11,0.31
...,...,...,...,...
4744,WFRC_8470,0.36,0.25,0.58
4745,WFRC_8471,0.60,0.48,0.75
4746,WFRC_8472,0.46,0.56,0.99
4747,WFRC_8473,0.68,0.57,0.76


VCGroupCode,SEGID,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr
0,0006_141.0,0.11,0.07,0.11
1,0006_146.9,0.13,0.08,0.12
2,0006_149.9,0.15,0.13,0.22
3,0006_150.6,0.15,0.14,0.23
4,0006_152.6,0.19,0.15,0.23
...,...,...,...,...
5031,WFRC_8470,0.36,0.25,0.38
5032,WFRC_8471,0.59,0.51,0.73
5033,WFRC_8472,0.45,0.65,1.01
5034,WFRC_8473,0.80,0.63,0.90


VCGroupCode,SEGID,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr
0,0006_141.0,0.11,0.07,0.11
1,0006_146.9,0.13,0.08,0.12
2,0006_149.9,0.14,0.13,0.23
3,0006_150.6,0.15,0.13,0.23
4,0006_152.6,0.18,0.15,0.24
...,...,...,...,...
5032,WFRC_8470,0.36,0.25,0.39
5033,WFRC_8471,0.59,0.51,0.73
5034,WFRC_8472,0.46,0.65,1.00
5035,WFRC_8473,0.81,0.62,0.90


In [343]:
dfs_VC_MaxDirFG_wVCGroups_Prd = []

for df_VC_MaxDirFG in dfs_VC_MaxDirFG:

    df_VC_MaxDirFG_wVCGroups_Prd = df_VC_MaxDirFG[df_VC_MaxDirFG['HRPCTOF'] == 'Prd'].copy()
    
    df_VC_MaxDirFG_wVCGroups_Prd['VCGroupCode'] = df_VC_MaxDirFG_wVCGroups_Prd['VCGroupCode'] + "Prd"
    
    df_VC_MaxDirFG_wVCGroups_Prd = df_VC_MaxDirFG_wVCGroups_Prd[['VCGroupCode','SEGID','PrdVCMax']]
    
    df_VC_MaxDirFG_wVCGroups_Prd = df_VC_MaxDirFG_wVCGroups_Prd.round({'PrdVC': 2})
    
    #df_VC_MaxDirFG_wVCGroups_Prd = df_VC_MaxDirFG_wVCGroups_Prd.pivot(index=('SEGID','ForecastAADT'), columns="VCGroupCode",values='PrdVCMax')
    df_VC_MaxDirFG_wVCGroups_Prd = df_VC_MaxDirFG_wVCGroups_Prd.pivot(index=('SEGID'), columns="VCGroupCode",values='PrdVCMax')
    
    df_VC_MaxDirFG_wVCGroups_Prd = df_VC_MaxDirFG_wVCGroups_Prd.reset_index()
    
    dfs_VC_MaxDirFG_wVCGroups_Prd.append(df_VC_MaxDirFG_wVCGroups_Prd)

    display(df_VC_MaxDirFG_wVCGroups_Prd)

VCGroupCode,SEGID,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.06,0.04,0.06
1,0006_146.9,0.07,0.05,0.06
2,0006_149.9,0.09,0.07,0.15
3,0006_150.6,0.09,0.07,0.14
4,0006_152.6,0.10,0.07,0.15
...,...,...,...,...
4314,WFRC_8461,0.15,0.12,0.18
4315,WFRC_8466,0.28,0.16,0.34
4316,WFRC_8467,0.42,0.25,0.55
4317,WFRC_8473,0.47,0.43,0.64


VCGroupCode,SEGID,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.07,0.05,0.07
1,0006_146.9,0.08,0.05,0.07
2,0006_149.9,0.10,0.07,0.17
3,0006_150.6,0.10,0.08,0.16
4,0006_152.6,0.11,0.08,0.17
...,...,...,...,...
4381,WFRC_8461,0.15,0.12,0.36
4382,WFRC_8466,0.32,0.18,0.38
4383,WFRC_8467,0.52,0.30,0.66
4384,WFRC_8473,0.47,0.43,0.64


VCGroupCode,SEGID,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.07,0.05,0.07
1,0006_146.9,0.08,0.05,0.07
2,0006_149.9,0.11,0.07,0.17
3,0006_150.6,0.11,0.08,0.16
4,0006_152.6,0.12,0.08,0.16
...,...,...,...,...
4573,WFRC_8466,0.33,0.19,0.40
4574,WFRC_8467,0.52,0.32,0.71
4575,WFRC_8471,0.39,0.30,0.52
4576,WFRC_8473,0.51,0.45,0.69


VCGroupCode,SEGID,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.08,0.06,0.09
1,0006_146.9,0.10,0.06,0.09
2,0006_149.9,0.16,0.09,0.18
3,0006_150.6,0.15,0.09,0.19
4,0006_152.6,0.17,0.10,0.19
...,...,...,...,...
4668,WFRC_8466,0.39,0.25,0.45
4669,WFRC_8467,0.54,0.38,0.63
4670,WFRC_8471,0.48,0.37,0.63
4671,WFRC_8473,0.57,0.50,0.72


VCGroupCode,SEGID,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.07,0.05,0.07
1,0006_146.9,0.08,0.05,0.07
2,0006_149.9,0.13,0.08,0.20
3,0006_150.6,0.13,0.08,0.19
4,0006_152.6,0.14,0.09,0.21
...,...,...,...,...
4382,WFRC_8461,0.22,0.15,0.48
4383,WFRC_8466,0.34,0.19,0.41
4384,WFRC_8467,0.59,0.33,0.69
4385,WFRC_8473,0.52,0.46,0.67


...

VCGroupCode,SEGID,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.09,0.06,0.09
1,0006_146.9,0.10,0.07,0.09
2,0006_149.9,0.15,0.09,0.20
3,0006_150.6,0.15,0.10,0.20
4,0006_152.6,0.18,0.12,0.25
...,...,...,...,...
4675,WFRC_8466,0.43,0.29,0.49
4676,WFRC_8467,0.53,0.40,0.62
4677,WFRC_8471,0.49,0.40,0.65
4678,WFRC_8473,0.62,0.54,0.80


VCGroupCode,SEGID,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.07,0.05,0.08
1,0006_146.9,0.08,0.05,0.08
2,0006_149.9,0.10,0.07,0.15
3,0006_150.6,0.10,0.07,0.15
4,0006_152.6,0.12,0.08,0.15
...,...,...,...,...
4605,WFRC_8466,0.34,0.19,0.41
4606,WFRC_8467,0.52,0.31,0.64
4607,WFRC_8471,0.38,0.32,0.53
4608,WFRC_8473,0.51,0.45,0.68


VCGroupCode,SEGID,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.09,0.06,0.09
1,0006_146.9,0.10,0.06,0.09
2,0006_149.9,0.11,0.09,0.34
3,0006_150.6,0.11,0.09,0.33
4,0006_152.6,0.12,0.09,0.29
...,...,...,...,...
4744,WFRC_8470,0.30,0.22,0.50
4745,WFRC_8471,0.50,0.42,0.67
4746,WFRC_8472,0.38,0.48,0.91
4747,WFRC_8473,0.56,0.48,0.70


VCGroupCode,SEGID,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.09,0.06,0.10
1,0006_146.9,0.10,0.07,0.11
2,0006_149.9,0.12,0.12,0.21
3,0006_150.6,0.12,0.12,0.21
4,0006_152.6,0.15,0.13,0.22
...,...,...,...,...
5031,WFRC_8470,0.30,0.22,0.33
5032,WFRC_8471,0.49,0.44,0.66
5033,WFRC_8472,0.37,0.55,0.93
5034,WFRC_8473,0.66,0.53,0.83


VCGroupCode,SEGID,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.09,0.06,0.10
1,0006_146.9,0.10,0.07,0.11
2,0006_149.9,0.12,0.11,0.21
3,0006_150.6,0.12,0.11,0.22
4,0006_152.6,0.15,0.13,0.22
...,...,...,...,...
5032,WFRC_8470,0.30,0.22,0.34
5033,WFRC_8471,0.49,0.44,0.66
5034,WFRC_8472,0.38,0.56,0.92
5035,WFRC_8473,0.67,0.53,0.83


In [344]:
df_Scenarios

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Base19,2019 Base,2019,Base,v901_SE19_Net19_Summary_SEGID_Detailed.csv,No,NaN,NaN
1,Base23,2023 Base,2023,Base,v901_SE23_Net23_Summary_SEGID_Detailed.csv,Yes,Base19,NaN
2,TIP28,2028 TIP Projects,2028,TIP,v901_TIP_SE28_Net28_Summary_SEGID_Detailed.csv,Yes,Base23,NaN
3,RTP32,2032 Fiscally-Constrained,2032,RTP,v901_RTP_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,NaN
4,RTP42,2042 Fiscally-Constrained,2042,RTP,v901_RTP_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,RTP32,NaN
...,...,...,...,...,...,...,...,...
11,42NB50,2050 SE on a 2042 Network,2050,NoBuild42,v901_NoBuild_SE50_Net42_Summary_SEGID_Detailed...,Yes,RTP42,RTP50
12,UF32,2032 Needs-Based,2032,Needs,v901_Needs_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,RTP32
13,UF42,2042 Needs-Based,2042,Needs,v901_Needs_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,UF32,RTP42


In [345]:
dfs_VC_MaxDirFG_wVCGroups = []

i=-1
for (idx, row) in df_Scenarios.iterrows():

    if row.loc['Process'] == 'Yes':

        i=i+1
        
        df_VC_MaxDirFG_wVCGroups = pd.DataFrame.merge(dfs_VC_MaxDirFG_wVCGroups_15[i],dfs_VC_MaxDirFG_wVCGroups_Hr[i],on='SEGID')
        df_VC_MaxDirFG_wVCGroups = pd.DataFrame.merge(df_VC_MaxDirFG_wVCGroups,dfs_VC_MaxDirFG_wVCGroups_Prd[i],on='SEGID')
        dfs_VC_MaxDirFG_wVCGroups.append(df_VC_MaxDirFG_wVCGroups)
        display(df_VC_MaxDirFG_wVCGroups)

VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.09,0.06,0.08,0.08,0.05,0.07,0.06,0.04,0.06
1,0006_146.9,0.10,0.06,0.08,0.09,0.05,0.07,0.07,0.05,0.06
2,0006_149.9,0.13,0.09,0.18,0.11,0.08,0.16,0.09,0.07,0.15
3,0006_150.6,0.12,0.09,0.17,0.11,0.08,0.15,0.09,0.07,0.14
4,0006_152.6,0.14,0.09,0.18,0.13,0.08,0.16,0.10,0.07,0.15
...,...,...,...,...,...,...,...,...,...,...
4314,WFRC_8461,0.20,0.16,0.24,0.18,0.14,0.21,0.15,0.12,0.18
4315,WFRC_8466,0.37,0.21,0.40,0.34,0.19,0.37,0.28,0.16,0.34
4316,WFRC_8467,0.55,0.32,0.67,0.51,0.29,0.62,0.42,0.25,0.55
4317,WFRC_8473,0.61,0.54,0.78,0.56,0.50,0.71,0.47,0.43,0.64


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.09,0.06,0.08,0.08,0.05,0.07,0.07,0.05,0.07
1,0006_146.9,0.11,0.07,0.09,0.10,0.06,0.08,0.08,0.05,0.07
2,0006_149.9,0.14,0.10,0.21,0.12,0.09,0.18,0.10,0.07,0.17
3,0006_150.6,0.14,0.10,0.20,0.12,0.09,0.18,0.10,0.08,0.16
4,0006_152.6,0.16,0.10,0.21,0.14,0.09,0.18,0.11,0.08,0.17
...,...,...,...,...,...,...,...,...,...,...
4381,WFRC_8461,0.20,0.14,0.43,0.18,0.13,0.40,0.15,0.12,0.36
4382,WFRC_8466,0.42,0.23,0.45,0.39,0.21,0.41,0.32,0.18,0.38
4383,WFRC_8467,0.68,0.38,0.79,0.63,0.35,0.73,0.52,0.30,0.66
4384,WFRC_8473,0.62,0.55,0.75,0.57,0.50,0.69,0.47,0.43,0.64


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.10,0.07,0.09,0.09,0.06,0.08,0.07,0.05,0.07
1,0006_146.9,0.11,0.07,0.09,0.10,0.06,0.08,0.08,0.05,0.07
2,0006_149.9,0.15,0.10,0.20,0.13,0.08,0.18,0.11,0.07,0.17
3,0006_150.6,0.15,0.10,0.20,0.13,0.09,0.18,0.11,0.08,0.16
4,0006_152.6,0.17,0.11,0.19,0.15,0.10,0.17,0.12,0.08,0.16
...,...,...,...,...,...,...,...,...,...,...
4573,WFRC_8466,0.44,0.25,0.48,0.40,0.23,0.44,0.33,0.19,0.40
4574,WFRC_8467,0.68,0.40,0.86,0.63,0.36,0.79,0.52,0.32,0.71
4575,WFRC_8471,0.51,0.38,0.63,0.47,0.35,0.58,0.39,0.30,0.52
4576,WFRC_8473,0.67,0.57,0.82,0.62,0.52,0.75,0.51,0.45,0.69


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.12,0.08,0.12,0.10,0.07,0.10,0.08,0.06,0.09
1,0006_146.9,0.13,0.08,0.12,0.12,0.07,0.10,0.10,0.06,0.09
2,0006_149.9,0.21,0.11,0.22,0.19,0.10,0.20,0.16,0.09,0.18
3,0006_150.6,0.21,0.12,0.23,0.19,0.10,0.20,0.15,0.09,0.19
4,0006_152.6,0.23,0.13,0.24,0.20,0.11,0.21,0.17,0.10,0.19
...,...,...,...,...,...,...,...,...,...,...
4668,WFRC_8466,0.51,0.32,0.53,0.47,0.29,0.48,0.39,0.25,0.45
4669,WFRC_8467,0.70,0.48,0.76,0.65,0.44,0.70,0.54,0.38,0.63
4670,WFRC_8471,0.63,0.46,0.76,0.58,0.43,0.70,0.48,0.37,0.63
4671,WFRC_8473,0.74,0.64,0.85,0.68,0.59,0.78,0.57,0.50,0.72


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.10,0.07,0.09,0.09,0.06,0.08,0.07,0.05,0.07
1,0006_146.9,0.11,0.07,0.09,0.10,0.06,0.08,0.08,0.05,0.07
2,0006_149.9,0.18,0.11,0.24,0.16,0.09,0.21,0.13,0.08,0.20
3,0006_150.6,0.18,0.11,0.24,0.16,0.09,0.21,0.13,0.08,0.19
4,0006_152.6,0.20,0.12,0.25,0.17,0.10,0.22,0.14,0.09,0.21
...,...,...,...,...,...,...,...,...,...,...
4382,WFRC_8461,0.28,0.18,0.58,0.26,0.17,0.53,0.22,0.15,0.48
4383,WFRC_8466,0.45,0.25,0.48,0.41,0.23,0.44,0.34,0.19,0.41
4384,WFRC_8467,0.77,0.42,0.84,0.71,0.39,0.77,0.59,0.33,0.69
4385,WFRC_8473,0.68,0.59,0.79,0.63,0.54,0.73,0.52,0.46,0.67


...

VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.12,0.08,0.11,0.11,0.07,0.10,0.09,0.06,0.09
1,0006_146.9,0.14,0.09,0.11,0.12,0.08,0.10,0.10,0.07,0.09
2,0006_149.9,0.21,0.12,0.25,0.18,0.11,0.22,0.15,0.09,0.20
3,0006_150.6,0.20,0.13,0.25,0.18,0.11,0.22,0.15,0.10,0.20
4,0006_152.6,0.25,0.16,0.30,0.22,0.14,0.26,0.18,0.12,0.25
...,...,...,...,...,...,...,...,...,...,...
4675,WFRC_8466,0.56,0.37,0.58,0.52,0.34,0.53,0.43,0.29,0.49
4676,WFRC_8467,0.69,0.50,0.75,0.64,0.46,0.69,0.53,0.40,0.62
4677,WFRC_8471,0.64,0.50,0.79,0.59,0.46,0.72,0.49,0.40,0.65
4678,WFRC_8473,0.82,0.68,0.91,0.76,0.63,0.86,0.62,0.54,0.80


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.10,0.07,0.10,0.09,0.06,0.09,0.07,0.05,0.08
1,0006_146.9,0.11,0.07,0.10,0.10,0.06,0.09,0.08,0.05,0.08
2,0006_149.9,0.14,0.09,0.19,0.13,0.08,0.16,0.10,0.07,0.15
3,0006_150.6,0.14,0.10,0.19,0.13,0.08,0.17,0.10,0.07,0.15
4,0006_152.6,0.16,0.11,0.18,0.14,0.09,0.16,0.12,0.08,0.15
...,...,...,...,...,...,...,...,...,...,...
4605,WFRC_8466,0.44,0.24,0.48,0.41,0.22,0.44,0.34,0.19,0.41
4606,WFRC_8467,0.68,0.39,0.77,0.62,0.36,0.71,0.52,0.31,0.64
4607,WFRC_8471,0.50,0.41,0.64,0.46,0.37,0.59,0.38,0.32,0.53
4608,WFRC_8473,0.67,0.57,0.81,0.61,0.53,0.74,0.51,0.45,0.68


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.12,0.08,0.11,0.10,0.07,0.10,0.09,0.06,0.09
1,0006_146.9,0.13,0.08,0.12,0.12,0.07,0.10,0.10,0.06,0.09
2,0006_149.9,0.15,0.11,0.41,0.13,0.10,0.36,0.11,0.09,0.34
3,0006_150.6,0.15,0.12,0.40,0.13,0.10,0.36,0.11,0.09,0.33
4,0006_152.6,0.16,0.12,0.35,0.14,0.11,0.31,0.12,0.09,0.29
...,...,...,...,...,...,...,...,...,...,...
4744,WFRC_8470,0.41,0.28,0.65,0.36,0.25,0.58,0.30,0.22,0.50
4745,WFRC_8471,0.65,0.52,0.81,0.60,0.48,0.75,0.50,0.42,0.67
4746,WFRC_8472,0.50,0.61,1.02,0.46,0.56,0.99,0.38,0.48,0.91
4747,WFRC_8473,0.74,0.61,0.83,0.68,0.57,0.76,0.56,0.48,0.70


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.13,0.08,0.13,0.11,0.07,0.11,0.09,0.06,0.10
1,0006_146.9,0.14,0.09,0.13,0.13,0.08,0.12,0.10,0.07,0.11
2,0006_149.9,0.17,0.15,0.26,0.15,0.13,0.22,0.12,0.12,0.21
3,0006_150.6,0.17,0.15,0.26,0.15,0.14,0.23,0.12,0.12,0.21
4,0006_152.6,0.21,0.17,0.27,0.19,0.15,0.23,0.15,0.13,0.22
...,...,...,...,...,...,...,...,...,...,...
5031,WFRC_8470,0.41,0.28,0.43,0.36,0.25,0.38,0.30,0.22,0.33
5032,WFRC_8471,0.64,0.55,0.80,0.59,0.51,0.73,0.49,0.44,0.66
5033,WFRC_8472,0.49,0.70,1.03,0.45,0.65,1.01,0.37,0.55,0.93
5034,WFRC_8473,0.87,0.68,0.95,0.80,0.63,0.90,0.66,0.53,0.83


VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.13,0.08,0.13,0.11,0.07,0.11,0.09,0.06,0.10
1,0006_146.9,0.14,0.09,0.13,0.13,0.08,0.12,0.10,0.07,0.11
2,0006_149.9,0.16,0.14,0.26,0.14,0.13,0.23,0.12,0.11,0.21
3,0006_150.6,0.17,0.15,0.26,0.15,0.13,0.23,0.12,0.11,0.22
4,0006_152.6,0.21,0.17,0.27,0.18,0.15,0.24,0.15,0.13,0.22
...,...,...,...,...,...,...,...,...,...,...
5032,WFRC_8470,0.41,0.29,0.44,0.36,0.25,0.39,0.30,0.22,0.34
5033,WFRC_8471,0.64,0.55,0.80,0.59,0.51,0.73,0.49,0.44,0.66
5034,WFRC_8472,0.50,0.71,1.02,0.46,0.65,1.00,0.38,0.56,0.92
5035,WFRC_8473,0.88,0.68,0.94,0.81,0.62,0.90,0.67,0.53,0.83


# Export final tables to CSV

In [346]:
df_Scenarios.to_csv(os.path.join(dirResults, r'scenarios.csv'),index=False)

In [347]:
if export_results: 
    df_Scenarios.to_csv(os.path.join(dirResults, r'scenarios.csv'),index=False)
    df_VCGroup_DOWPkHr.to_csv(os.path.join(dirResults, r'vcgroups_dowpeak.csv'),index=False)
    df_VCGroup_Seasons.to_csv(os.path.join(dirResults, r'vcgroups_seasons.csv'),index=False)
    df_VCGroups.to_csv(os.path.join(dirResults, r'vcgroups.csv'),index=False)

In [348]:
if export_results: 

    df_Scenarios_toExport = df_Scenarios[df_Scenarios['Process']=='Yes']
    
    i=0
    for (idx, row) in df_Scenarios_toExport.iterrows():    
    
        filename = os.path.join(dirResults,"3_VC_MaxDirFG_wVCGroups_" + row.loc['ScenarioCode'] + '.csv')
        display('Exporting ' + filename)
        dfs_VC_MaxDirFG_wVCGroups[i].to_csv(filename, index=False)

        filename = os.path.join(dirResults,"2_VC_MaxDirFG_" + row.loc['ScenarioCode'] + '.csv')
        display('Exporting ' + filename)
        dfs_VC_MaxDirFG[i].to_csv(filename, index=False)

        filename = os.path.join(dirResults,"1_VC_" + row.loc['ScenarioCode'] + '.csv')
        display('Exporting ' + filename)
        dfs_VC[i].to_csv(filename, index=False)
        i=i+1

'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\3_VC_MaxDirFG_wVCGroups_Base23.csv'

'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\2_VC_MaxDirFG_Base23.csv'

'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\1_VC_Base23.csv'

'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\3_VC_MaxDirFG_wVCGroups_TIP28.csv'

'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\2_VC_MaxDirFG_TIP28.csv'

...

'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\2_VC_MaxDirFG_UF50.csv'

'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\1_VC_UF50.csv'

'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\3_VC_MaxDirFG_wVCGroups_UF50_MAG.csv'

'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\2_VC_MaxDirFG_UF50_MAG.csv'

'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\1_VC_UF50_MAG.csv'

In [349]:
for (idx, row) in df_Scenarios_toExport.iterrows():
    display(row.loc['ScenarioCode'])

'Base23'

'TIP28'

'RTP32'

'RTP42'

'28NB32'

...

'42NB50'

'UF32'

'UF42'

'UF50'

'UF50_MAG'

In [350]:
df = dfs_VC_MaxDirFG_wVCGroups[0]
df[df['SEGID']=='0079_003.0']
df

VCGroupCode,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.09,0.06,0.08,0.08,0.05,0.07,0.06,0.04,0.06
1,0006_146.9,0.10,0.06,0.08,0.09,0.05,0.07,0.07,0.05,0.06
2,0006_149.9,0.13,0.09,0.18,0.11,0.08,0.16,0.09,0.07,0.15
3,0006_150.6,0.12,0.09,0.17,0.11,0.08,0.15,0.09,0.07,0.14
4,0006_152.6,0.14,0.09,0.18,0.13,0.08,0.16,0.10,0.07,0.15
...,...,...,...,...,...,...,...,...,...,...
4314,WFRC_8461,0.20,0.16,0.24,0.18,0.14,0.21,0.15,0.12,0.18
4315,WFRC_8466,0.37,0.21,0.40,0.34,0.19,0.37,0.28,0.16,0.34
4316,WFRC_8467,0.55,0.32,0.67,0.51,0.29,0.62,0.42,0.25,0.55
4317,WFRC_8473,0.61,0.54,0.78,0.56,0.50,0.71,0.47,0.43,0.64


In [351]:
df_Scenarios_toProcess

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
1,Base23,2023 Base,2023,Base,v901_SE23_Net23_Summary_SEGID_Detailed.csv,Yes,Base19,NaN
2,TIP28,2028 TIP Projects,2028,TIP,v901_TIP_SE28_Net28_Summary_SEGID_Detailed.csv,Yes,Base23,NaN
3,RTP32,2032 Fiscally-Constrained,2032,RTP,v901_RTP_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,NaN
4,RTP42,2042 Fiscally-Constrained,2042,RTP,v901_RTP_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,RTP32,NaN
6,28NB32,2032 SE on a 2028 Network,2032,NoBuild28,v901_NoBuild_SE32_Net28_Summary_SEGID_Detailed...,Yes,TIP28,RTP32
...,...,...,...,...,...,...,...,...
11,42NB50,2050 SE on a 2042 Network,2050,NoBuild42,v901_NoBuild_SE50_Net42_Summary_SEGID_Detailed...,Yes,RTP42,RTP50
12,UF32,2032 Needs-Based,2032,Needs,v901_Needs_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,RTP32
13,UF42,2042 Needs-Based,2042,Needs,v901_Needs_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,UF32,RTP42


# Exports for WebApp

In [352]:
if export_results:
    
    df_Scenarios_json = df_Scenarios.copy()
    df_Scenarios_json = df_Scenarios_json[['ScenarioCode','ScenarioName']]
    df_Scenarios_json.columns = ('value','label')
    df_Scenarios_json['filename'] = "VC_" + df_Scenarios_json['value'] + '.json'
    
    df_VCGroup_Seasons_json = df_VCGroup_Seasons.copy()
    df_VCGroup_Seasons_json = df_VCGroup_Seasons_json[['SeasonCode','SeasonName','SeasonType']]
    df_VCGroup_Seasons_json.columns = ('value','label','SeasonType')
        
    df_VCGroup_DOWPkHr_json = df_VCGroup_DOWPkHr.copy()
    df_VCGroup_DOWPkHr_json = df_VCGroup_DOWPkHr_json[['DOWPkCode','DOWPkName']]
    df_VCGroup_DOWPkHr_json.columns = ('value','label')

    df_Scenarios_json.to_json(os.path.join(dirResults, r'WebAppData\scenarios.json'),orient='records')
    #df_Scenarios_json = df_Scenarios_json.reset_index()
    #df_Scenarios_json.to_json(os.path.join(dirResults, r'WebAppData\scenarios_test.json'))
    df_VCGroup_Seasons_json.to_json(os.path.join(dirResults, r'WebAppData\vcgroups_seasons.json'),orient='records')
    df_VCGroup_DOWPkHr_json.to_json(os.path.join(dirResults, r'WebAppData\vcgroups_dowpeaks.json'),orient='records')
    

In [353]:
df_Scenarios_json

,value,label,filename
ScenarioID,,,
0,Base19,2019 Base,VC_Base19.json
1,Base23,2023 Base,VC_Base23.json
2,TIP28,2028 TIP Projects,VC_TIP28.json
3,RTP32,2032 Fiscally-Constrained,VC_RTP32.json
4,RTP42,2042 Fiscally-Constrained,VC_RTP42.json
...,...,...,...
11,42NB50,2050 SE on a 2042 Network,VC_42NB50.json
12,UF32,2032 Needs-Based,VC_UF32.json
13,UF42,2042 Needs-Based,VC_UF42.json


In [354]:
if export_results:
    
    df_Scenarios_toExport = df_Scenarios_toProcess.reset_index()
    
    for index, df_VC_MaxDirFG_wVCGroups in enumerate(dfs_VC_MaxDirFG_wVCGroups):    

        filename = "WebAppData\VC_" + df_Scenarios_toExport['ScenarioCode'].iloc[index] + '.json'
        #display(filename)

        #df_Peak_VC_MaxDirFG_wVCGroups = df_Peak_VC_MaxDirFG_wVCGroups.rename(columns={"SEGID": 'S'})

        #export to csv in data-processed
        df_VC_MaxDirFG_wVCGroups.to_json(os.path.join(dirResults, filename),orient='records')

        display('Done exporting ' + filename)

'Done exporting WebAppData\\VC_Base23.json'

'Done exporting WebAppData\\VC_TIP28.json'

'Done exporting WebAppData\\VC_RTP32.json'

'Done exporting WebAppData\\VC_RTP42.json'

'Done exporting WebAppData\\VC_28NB32.json'

...

'Done exporting WebAppData\\VC_42NB50.json'

'Done exporting WebAppData\\VC_UF32.json'

'Done exporting WebAppData\\VC_UF42.json'

'Done exporting WebAppData\\VC_UF50.json'

'Done exporting WebAppData\\VC_UF50_MAG.json'

# Temp

In [355]:
df_Scenarios

,ScenarioCode,ScenarioName,ScenarioYear,ScenarioGroup,SegSummaryCSV,Process,PrvScenarioCode,AdjScenarioCode
ScenarioID,,,,,,,,
0,Base19,2019 Base,2019,Base,v901_SE19_Net19_Summary_SEGID_Detailed.csv,No,NaN,NaN
1,Base23,2023 Base,2023,Base,v901_SE23_Net23_Summary_SEGID_Detailed.csv,Yes,Base19,NaN
2,TIP28,2028 TIP Projects,2028,TIP,v901_TIP_SE28_Net28_Summary_SEGID_Detailed.csv,Yes,Base23,NaN
3,RTP32,2032 Fiscally-Constrained,2032,RTP,v901_RTP_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,NaN
4,RTP42,2042 Fiscally-Constrained,2042,RTP,v901_RTP_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,RTP32,NaN
...,...,...,...,...,...,...,...,...
11,42NB50,2050 SE on a 2042 Network,2050,NoBuild42,v901_NoBuild_SE50_Net42_Summary_SEGID_Detailed...,Yes,RTP42,RTP50
12,UF32,2032 Needs-Based,2032,Needs,v901_Needs_SE32_Net32_Summary_SEGID_Detailed.csv,Yes,TIP28,RTP32
13,UF42,2042 Needs-Based,2042,Needs,v901_Needs_SE42_Net42_Summary_SEGID_Detailed.csv,Yes,UF32,RTP42


In [356]:
dfs_VC_MaxDirFG_wVCGroups =[]
df_TDMsf = []

display ('Scenario Codes:')

for (idx, row) in df_Scenarios.iterrows():    

    print(row.loc['ScenarioCode'])
    
    df_TDM = dfs_TDM[idx]
    
    df_TDMsf = df_TDM[['SEGID','FT']]
    
    df_TDMsfmax = df_TDMsf.groupby('SEGID', as_index=False).agg({'FT':[np.max]})
    df_TDMsfmax.columns = ['SEGID','FT']

    display(df_TDMsfmax)
    #df_TDMsf.append(df_TDMsf)
    
    df_VC_MaxDirFG_wVCGroups = pd.read_csv(os.path.join(dirResults, '3_VC_MaxDirFG_wVCGroups_' + row.loc['ScenarioCode'] + '.csv'))
    display(df_VC_MaxDirFG_wVCGroups)
    
    df_new = pd.DataFrame.merge(df_TDMsfmax,df_VC_MaxDirFG_wVCGroups,on=('SEGID'))
    display(df_new)
    
    filename = os.path.join(dirResults,r"temp\3_VC_MaxDirFG_wVCGroups_" + row.loc['ScenarioCode'] + '.csv')
    display('Exporting ' + filename)
    df_new.to_csv(filename,index=False)
    #dfs_VC_MaxDirFG_wVCGroups.append(dfs_Peak_VC_MaxDirFG_wVCGroups)


'Scenario Codes:'

Base19


,SEGID,FT
0,0006_141.0,2.0
1,0006_146.9,2.0
2,0006_149.9,2.0
3,0006_150.6,2.0
4,0006_152.6,3.0
...,...,...
4222,WFRC_8457,4.0
4223,WFRC_8461,5.0
4224,WFRC_8466,3.0
4225,WFRC_8473,5.0


,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.08,0.05,0.08,0.07,0.05,0.07,0.06,0.04,0.07
1,0006_146.9,0.09,0.06,0.09,0.08,0.05,0.08,0.07,0.04,0.07
2,0006_149.9,0.12,0.09,0.16,0.11,0.08,0.14,0.09,0.07,0.13
3,0006_150.6,0.12,0.09,0.16,0.10,0.08,0.14,0.08,0.07,0.13
4,0006_152.6,0.14,0.10,0.19,0.13,0.09,0.16,0.10,0.07,0.15
...,...,...,...,...,...,...,...,...,...,...
4222,WFRC_8457,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4223,WFRC_8461,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4224,WFRC_8466,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4225,WFRC_8473,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


,SEGID,FT,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,2.0,0.08,0.05,0.08,0.07,0.05,0.07,0.06,0.04,0.07
1,0006_146.9,2.0,0.09,0.06,0.09,0.08,0.05,0.08,0.07,0.04,0.07
2,0006_149.9,2.0,0.12,0.09,0.16,0.11,0.08,0.14,0.09,0.07,0.13
3,0006_150.6,2.0,0.12,0.09,0.16,0.10,0.08,0.14,0.08,0.07,0.13
4,0006_152.6,3.0,0.14,0.10,0.19,0.13,0.09,0.16,0.10,0.07,0.15
...,...,...,...,...,...,...,...,...,...,...,...
4222,WFRC_8457,4.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4223,WFRC_8461,5.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4224,WFRC_8466,3.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4225,WFRC_8473,5.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


...

UF50_MAG


,SEGID,FT
0,0006_141.0,2.0
1,0006_146.9,2.0
2,0006_149.9,2.0
3,0006_150.6,2.0
4,0006_152.6,3.0
...,...,...
5032,WFRC_8470,3.0
5033,WFRC_8471,4.0
5034,WFRC_8472,5.0
5035,WFRC_8473,5.0


,SEGID,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,0.13,0.08,0.13,0.11,0.07,0.11,0.09,0.06,0.10
1,0006_146.9,0.14,0.09,0.13,0.13,0.08,0.12,0.10,0.07,0.11
2,0006_149.9,0.16,0.14,0.26,0.14,0.13,0.23,0.12,0.11,0.21
3,0006_150.6,0.17,0.15,0.26,0.15,0.13,0.23,0.12,0.11,0.22
4,0006_152.6,0.21,0.17,0.27,0.18,0.15,0.24,0.15,0.13,0.22
...,...,...,...,...,...,...,...,...,...,...
5032,WFRC_8470,0.41,0.29,0.44,0.36,0.25,0.39,0.30,0.22,0.34
5033,WFRC_8471,0.64,0.55,0.80,0.59,0.51,0.73,0.49,0.44,0.66
5034,WFRC_8472,0.50,0.71,1.02,0.46,0.65,1.00,0.38,0.56,0.92
5035,WFRC_8473,0.88,0.68,0.94,0.81,0.62,0.90,0.67,0.53,0.83


,SEGID,FT,AnnWkAM15,AnnWkMD15,AnnWkPM15,AnnWkAMHr,AnnWkMDHr,AnnWkPMHr,AnnWkAMPrd,AnnWkMDPrd,AnnWkPMPrd
0,0006_141.0,2.0,0.13,0.08,0.13,0.11,0.07,0.11,0.09,0.06,0.10
1,0006_146.9,2.0,0.14,0.09,0.13,0.13,0.08,0.12,0.10,0.07,0.11
2,0006_149.9,2.0,0.16,0.14,0.26,0.14,0.13,0.23,0.12,0.11,0.21
3,0006_150.6,2.0,0.17,0.15,0.26,0.15,0.13,0.23,0.12,0.11,0.22
4,0006_152.6,3.0,0.21,0.17,0.27,0.18,0.15,0.24,0.15,0.13,0.22
...,...,...,...,...,...,...,...,...,...,...,...
5032,WFRC_8470,3.0,0.41,0.29,0.44,0.36,0.25,0.39,0.30,0.22,0.34
5033,WFRC_8471,4.0,0.64,0.55,0.80,0.59,0.51,0.73,0.49,0.44,0.66
5034,WFRC_8472,5.0,0.50,0.71,1.02,0.46,0.65,1.00,0.38,0.56,0.92
5035,WFRC_8473,5.0,0.88,0.68,0.94,0.81,0.62,0.90,0.67,0.53,0.83


'Exporting e:\\GitHub\\V-over-C-Calculations\\results\\temp\\3_VC_MaxDirFG_wVCGroups_UF50_MAG.csv'

In [357]:
df_TDMsfmax = df_TDMsf.groupby('SEGID', as_index=False).agg({'FT':[np.max]})
df_TDMsfmax.columns = ['SEGID','FT']
df_TDMsfmax

,SEGID,FT
0,0006_141.0,2.0
1,0006_146.9,2.0
2,0006_149.9,2.0
3,0006_150.6,2.0
4,0006_152.6,3.0
...,...,...
5032,WFRC_8470,3.0
5033,WFRC_8471,4.0
5034,WFRC_8472,5.0
5035,WFRC_8473,5.0


In [358]:
df_VCGroup_Seasons

,SeasonCode,SeasonName,SEASONGROUP,SeasonType
0,Ann,Annual,S00-Ann,Season


In [359]:
df_TDMsfmax

,SEGID,FT
0,0006_141.0,2.0
1,0006_146.9,2.0
2,0006_149.9,2.0
3,0006_150.6,2.0
4,0006_152.6,3.0
...,...,...
5032,WFRC_8470,3.0
5033,WFRC_8471,4.0
5034,WFRC_8472,5.0
5035,WFRC_8473,5.0
